Regression model code based upon https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/

Majority of the code adapted from [artificial data experiment with case removal.ipynb](https://colab.research.google.com/drive/13eN2qzSAB_wRqzzukBVjqcvt2GayzDCl?authuser=1#scrollTo=pRXm-C2PuCWe) 

# Data Preparation

In [2]:
import keras
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.utils import shuffle


Code adapted from https://www.kaggle.com/jshih7/car-price-prediction
to read data file and handle nominal attributes

also using a trick from https://stackoverflow.com/questions/18889588/create-dummies-from-column-with-multiple-values-in-pandas

Main Code

In [3]:
# Use some functions from tensorflow_docs
!pip install -q git+https://github.com/tensorflow/docs


     |████████████████████████████████| 1.0MB 4.3MB/s 


Model code adapted from https://machinelearningmastery.com/cost-sensitive-neural-network-for-imbalanced-classification/

https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/

In [4]:
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense

def build_DL_model(X_train):
  model = Sequential()
  model.add(Dense(128, input_dim=X_train.shape[1], kernel_initializer='he_uniform', activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(num_classes, activation='softmax'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

Adaptation ML-CDH models

Style 2: source problem + problem difference => solution difference

In [5]:
def build_adaptation_model(X_train_source_n_diff):
  model = Sequential()
  model.add(Dense(128, input_dim=len(X_train_source_n_diff[0]), kernel_initializer='he_uniform', activation='relu'))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(num_classes, activation='softmax'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

This provides a baseline

In [6]:
# from keras.wrappers.scikit_learn import KerasClassifier
# from sklearn.model_selection import KFold
# from sklearn.model_selection import cross_val_score

# estimator = KerasClassifier(build_fn=build_DL_model, epochs=200, batch_size=5, verbose=0)
# kfold = KFold(n_splits=10, shuffle=True)
# results = cross_val_score(estimator, X, y, cv=kfold)
# print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

The code above is commented out for efficiency

Baseline executation result: 

Baseline: 82.69% (3.34%)

# Testing full test set without removal

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
import matplotlib.pyplot as plt


def test_full_without_removal(EAC_adapt,pair_selection, pair_knowledge, verbose_param):
  #pure DL model
  dl_model = build_DL_model()
  # The patience parameter is the amount of epochs to check for improvement
  early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
  EPOCHS = 1000
  early_history = dl_model.fit(X_train, y_train, 
                      epochs=EPOCHS, validation_data=(X_test, y_test), verbose=0, 
                      callbacks=[early_stop, tfdocs.modeling.EpochDots()])
  
  # X_dl_train, X_dl_valid, y_dl_train, y_dl_valid = train_test_split(X_train_trim, y_train_trim, test_size=0.1, shuffle= True)
  # early_history = dl_model.fit(X_dl_train, y_dl_train, 
  #                     epochs=EPOCHS, validation_data=(X_dl_valid, y_dl_valid), verbose=verbose_param, 
  #                     callbacks=[early_stop, tfdocs.modeling.EpochDots()])


  y_test_pred = dl_model.predict(X_test)
  y_test_pred = tf.one_hot(tf.argmax(y_test_pred, 1), y_test_pred.shape[1])
  dl_loss = accuracy_score(y_test, y_test_pred)

  print("dl last loss: ",early_history.history['val_loss'][-1])

  #pure knn
  knn = KNeighborsClassifier(n_neighbors=3)
  knn.fit(X_train, y_train)
  y_test_pred = knn.predict(X_test)  
  knn_loss = accuracy_score(y_test, y_test_pred)

  #pure CBR retrieval (knn k=1), no need to retrain knn
  X_test_retrieved_index = knn.kneighbors(X_test, n_neighbors=1)[1]
  #we only select the best neighbor here!!!
  X_test_retrieved_index = X_test_retrieved_index[:,0]
  X_test_neighbors = X_train.iloc[X_test_retrieved_index]
  y_test_neighbors = y_train.iloc[X_test_retrieved_index]
  retrieval_loss = accuracy_score(y_test, y_test_neighbors)

  if(EAC_adapt==1):
    #EAC retrieval
    X_test_EAC_retrieved_index = knn.kneighbors(X_test, n_neighbors=3)[1]
    # print("X_test_EAC_retrieved_index")
    # print(X_test_EAC_retrieved_index)
    #bug fixing
    # X_test_EAC_neighbors = X_train.iloc[X_test_EAC_retrieved_index]
    # print("X_test_EAC_neighbors")
    # print(X_test_EAC_neighbors)

  #CBR adaptaion ML-CDH learning
  #pool for training pairs
  X_train_for_CDH = X_train
  y_train_for_CDH = y_train


  X_train_pairs_by_class = [[] for _ in range(num_classes)]
  y_train_pairs_by_class = [[] for _ in range(num_classes)]

  def addToPairList(source_y, X_train_pair, target_y):
    for src_class in range(num_classes):
      if source_y[src_class] == 1:
        X_train_pairs_by_class[src_class].append(X_train_pair)
        y_train_pairs_by_class[src_class].append(target_y)
        break

  #Assemble pairs
  if pair_selection == 1:#not updated
    
    knn_for_CDH = KNeighborsClassifier(n_neighbors=2)
    knn_for_CDH.fit(X_train_for_CDH, y_train_for_CDH)
    X_train_neighbors_index = knn_for_CDH.kneighbors(X_train_for_CDH)[1]
    #we only select the second best neighbor, the first best neighbor of a case will be itself
    X_train_neighbors_index = X_train_neighbors_index[:,1]
    X_train_neighbors = X_train_for_CDH[X_train_neighbors_index]
    #IMPORTANT NOTE: left side of difference should be target, right side is source (or the retrieved case)
    X_train_differences = X_train_for_CDH - X_train_neighbors
    y_train_neighbors = y_train_for_CDH[X_train_neighbors_index]
    y_train_differences = y_train_for_CDH - y_train_neighbors
    X_train_source_n_diff = np.concatenate((X_train_neighbors, X_train_differences),axis = 1)
  elif pair_selection == 2:#not updated
    X_train_for_CDH_index = [*range(len(X_train_for_CDH))]
    np.random.shuffle(X_train_for_CDH_index) #shuffle in place
    X_train_neighbors = X_train_for_CDH[X_train_for_CDH_index]
    y_train_neighbors = y_train_for_CDH[X_train_for_CDH_index]
    X_train_differences = X_train_for_CDH - X_train_neighbors
    y_train_differences = y_train_for_CDH - y_train_neighbors
    X_train_source_n_diff = np.concatenate((X_train_neighbors, X_train_differences),axis = 1)
  elif pair_selection == 12 or pair_selection==124:
    #1
    knn_for_CDH = KNeighborsClassifier(n_neighbors=2)
    knn_for_CDH.fit(X_train_for_CDH, y_train_for_CDH)
    X_train_neighbors_index = knn_for_CDH.kneighbors(X_train_for_CDH)[1]
    #we only select the second best neighbor, the first best neighbor of a case will be itself
    X_train_neighbors_index = X_train_neighbors_index[:,1]
    X_train_neighbors1 = X_train_for_CDH.iloc[X_train_neighbors_index]
    #IMPORTANT NOTE: left side of difference should be target, right side is source (or the retrieved case)
    y_train_neighbors = y_train_for_CDH.iloc[X_train_neighbors_index]

    for i in range(len(X_train_for_CDH.index)):
      # print(X_train_for_CDH.shape)
      # print(pd.concat([X_train_for_CDH.iloc[[i]], X_train_neighbors1.iloc[[i]]],axis = 1).shape)
      # print(pd.concat([X_train_for_CDH.iloc[[i]], X_train_neighbors1.iloc[[i]]],axis = 1))
      # print(pd.concat([X_train_for_CDH.iloc[i], X_train_neighbors1.iloc[i]],axis = 0).shape)
      # print(pd.concat([X_train_for_CDH.iloc[i], X_train_neighbors1.iloc[i]],ignore_index=True))
      # print(pd.concat([X_train_for_CDH.iloc[i], X_train_neighbors1.iloc[i]],ignore_index=True).shape)
      #TODO, maybe use series version? 
      # addToPairList(y_train_for_CDH.iloc[i], pd.concat([X_train_for_CDH.iloc[[i]], X_train_neighbors1.iloc[[i]]],axis = 1), y_train_neighbors.iloc[i])
      addToPairList(y_train_for_CDH.iloc[i], pd.concat([X_train_for_CDH.iloc[i], X_train_neighbors1.iloc[i]],ignore_index=True), y_train_neighbors.iloc[i])
    #2
    X_train_neighbors2_left = 0
    X_train_neighbors2_right = 0
    y_train_neighbors2_left = 0
    y_train_neighbors2_right = 0
    if pair_selection == 12:#not updated
      X_train_for_CDH_index = [*range(len(X_train_for_CDH))]
      np.random.shuffle(X_train_for_CDH_index) #shuffle in place
      X_train_neighbors2 = X_train_for_CDH[X_train_for_CDH_index]
      y_train_neighbors = y_train_for_CDH[X_train_for_CDH_index]
      X_train_differences2 = X_train_for_CDH - X_train_neighbors2
      y_train_differences2 = y_train_for_CDH - y_train_neighbors
    elif pair_selection ==124:
      left_index = np.random.choice(len(X_train_for_CDH), random_pairs_count)
      right_index = np.random.choice(len(X_train_for_CDH), random_pairs_count)
      X_train_neighbors2_left = X_train_for_CDH.iloc[left_index]
      y_train_neighbors2_left = y_train_for_CDH.iloc[left_index]
      X_train_neighbors2_right = X_train_for_CDH.iloc[right_index]
      y_train_neighbors2_right = y_train_for_CDH.iloc[right_index]
    for i in range(len(X_train_neighbors2_left.index)):
      addToPairList(y_train_neighbors2_left.iloc[i], pd.concat([X_train_neighbors2_left.iloc[i], X_train_neighbors2_right.iloc[i]],ignore_index=True),y_train_neighbors2_right.iloc[i])
    # X_train_pairs1 = np.concatenate((X_train_for_CDH, X_train_neighbors1), axis = 1)
    # X_train_pairs2 = np.concatenate((X_train_neighbors2_left, X_train_neighbors2_right), axis = 1)
    # X_train_pairs = np.concatenate((X_train_pairs1,X_train_pairs2), axis = 0)
    # y_train_pairs = np.concatenate((y_train_neighbors, y_train_neighbors2), axis= 0)
    

  
  #converting pairs data to nparray
  for i in range(num_classes):
    X_train_pairs_by_class[i] = np.asarray(X_train_pairs_by_class[i]) 
    y_train_pairs_by_class[i] = np.asarray(y_train_pairs_by_class[i])
  # print("test1vvfdi:")
  # print(X_train_pairs_by_class[0][0])
  # print("shape", X_train_pairs_by_class[0][0].shape)
  # print(y_train_pairs_by_class[0][0])

  #training adapt models
  adapt_models = [None for _ in range(num_classes)]
  for i in range(num_classes):
    adapt_model = None
    #select training pairs
    X_train_pairs = X_train_pairs_by_class[i]
    y_train_pairs = y_train_pairs_by_class[i]

    # print("Debug 389ngfv")
    # print(len(X_train_pairs))
    # print(len(X_train_pairs[0]))
    # print(len(y_train_pairs))
    # print("length is",len(X_train_pairs[0]))
    adapt_model = build_adaptation_model(X_train_pairs)
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=verbose_param)
    EPOCHS = 1000
    X_adapt_train, X_adapt_valid, y_adapt_train, y_adapt_valid = train_test_split(X_train_pairs, y_train_pairs, test_size=0.05, shuffle= True)
    early_history = adapt_model.fit(X_adapt_train, y_adapt_train, 
                        epochs=EPOCHS, validation_data=(X_adapt_valid, y_adapt_valid), verbose=verbose_param, 
                        callbacks=[early_stop, tfdocs.modeling.EpochDots()])

    print("adapt model last loss: ",early_history.history['val_loss'][-1])
    adapt_models[i] = adapt_model
  #CBR retrieval + adaptation
  # X_test_differences = X_test - X_test_neighbors
  y_adapt_pred = []

  for i in range(len(X_test_neighbors.index)):
    for src_class in range(num_classes):
      if y_test_neighbors.iloc[i][src_class] == 1: #column label might be an issue. TOOD
        queryForAdapt = np.asarray([pd.concat([X_test_neighbors.iloc[i], X_test.iloc[i]],ignore_index=True)])
        y_pred = adapt_models[src_class].predict(queryForAdapt)
        y_adapt_pred.append(y_pred[0])
        continue

  y_adapt_pred = np.asarray(y_adapt_pred)
  y_adapt_pred = tf.one_hot(tf.argmax(y_adapt_pred, 1), y_adapt_pred.shape[1])

  retrieval_N_adapt_loss = accuracy_score(y_test, y_adapt_pred)

  #EAC + CDH
  retrieval_N_EAC_adapt_loss = -1
  if(EAC_adapt==1):
    y_adapt_pred = []
    for i in range(len(X_test_EAC_retrieved_index)):
      #for one query
      # voting = [0 for _ in range(num_classes)]
      voting = np.zeros((num_classes,), dtype=int)
      for j in range(len(X_test_EAC_retrieved_index[i])):
        retrieved_index = X_test_EAC_retrieved_index[i][j]
        for src_class in range(num_classes):
          if(y_train.iloc[retrieved_index][src_class] == 1):
            queryForAdapt = np.asarray([pd.concat([X_train.iloc[retrieved_index], X_test.iloc[i]],ignore_index=True)])
            y_pred = adapt_models[src_class].predict(queryForAdapt)
            voting = np.add(voting, y_pred[0])
      y_adapt_pred.append(voting)

    y_adapt_pred = np.asarray(y_adapt_pred)
    y_adapt_pred = tf.one_hot(tf.argmax(y_adapt_pred, 1), y_adapt_pred.shape[1])

    retrieval_N_EAC_adapt_loss = accuracy_score(y_test, y_adapt_pred)
  #normal CDH
  cdh_knns = [KNeighborsClassifier(n_neighbors=1) for _ in range(num_classes)]
  # cdh_knn = KNeighborsClassifier(n_neighbors=1)
  # might need multiple knn here , one for each class.
  for class_index in range(num_classes):
    X_train_pairs = X_train_pairs_by_class[class_index]
    y_train_pairs = y_train_pairs_by_class[class_index]
    cdh_knns[class_index].fit(X_train_pairs, y_train_pairs)
  # y_diff_pred = knn.predict(pd.concat([X_test_neighbors, X_test],axis = 1))  
  y_diff_pred = []
  for i in range(len(X_test_neighbors.index)):
    for src_class in range(num_classes):
      if y_test_neighbors.iloc[i][src_class] == 1: #column label might be an issue. TOOD
        queryForAdapt = np.asarray([pd.concat([X_test_neighbors.iloc[i], X_test.iloc[i]],ignore_index=True)])
        y_pred = cdh_knns[src_class].predict(queryForAdapt)
        y_diff_pred.append(y_pred[0])
        continue
  answer_pred_adapted =  y_diff_pred
  answer_pred_adapted = np.asarray(answer_pred_adapted)
  # print("normal Cdh",answer_pred_adapted)
  normal_cdh_loss = accuracy_score(y_test, answer_pred_adapted)

  #CBR retrieval + adaptation
  # X_test_differences = X_test - X_test_neighbors
  # y_adapt_pred = adapt_model.predict(X_test_differences)
  # y_test_pred = y_test_neighbors + y_adapt_pred
  print("")
  print("dl_loss", dl_loss)
  print("knn_loss", knn_loss)
  print("retrieval_loss", retrieval_loss)
  print("retrieval_N_adapt_loss", retrieval_N_adapt_loss)
  print("CBR normal CDH loss", normal_cdh_loss)
  if(EAC_adapt==1):
    print("retrieval_N_EAC_adapt_loss loss", retrieval_N_EAC_adapt_loss)
  # return (dl_model, knn, adapt_model)
  return (dl_loss, knn_loss, retrieval_loss, retrieval_N_adapt_loss, normal_cdh_loss, retrieval_N_EAC_adapt_loss)

#Separate the training and testing



In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#revamped feature config
#EAC_adapt:0, no EAC; 1 EAC; 2 C2C-EAC
#pair_selection: 1, neighbor; 4, random pairs; 5, C2C pairs
#C2C IS ONLY USABLE WHEN EAC_adapt contains 2 and pair_selection contains 5 
def train_models(X_train,y_train, EAC_adapt,pair_selection, pair_knowledge, verbose_param):
  #pure DL model
  dl_model = build_DL_model(X_train)
  # The patience parameter is the amount of epochs to check for improvement
  early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
  EPOCHS = 1000
  X_dl_train, X_dl_valid, y_dl_train, y_dl_valid = train_test_split(X_train, y_train, test_size=0.1, shuffle= True)
  early_history = dl_model.fit(X_dl_train, y_dl_train, 
                      epochs=EPOCHS, validation_data=(X_dl_valid, y_dl_valid), verbose=verbose_param, 
                      callbacks=[early_stop, tfdocs.modeling.EpochDots()])

  # plotter.plot({'Early Stopping': early_history}, metric = "mae")
  # plt.ylabel('MAE')
  print("dl last loss: ",early_history.history['val_loss'][-1])


  #pure knn
  knn = KNeighborsClassifier(n_neighbors=3)
  knn.fit(X_train, y_train)

  #knn by class
  X_train_by_class = [[] for _ in range(num_classes)]
  y_train_by_class = [[] for _ in range(num_classes)]
  knn_by_class= [0 for _ in range(num_classes) ]
  if("2" in EAC_adapt and "5" in pair_selection):
    for i in range(len(y_train.index)):
      for src_class in range(num_classes):
        if y_train.iloc[i][src_class] == 1:
          X_train_by_class[src_class].append(X_train.iloc[i])
          y_train_by_class[src_class].append(y_train.iloc[i])
          break
    for src_class in range(num_classes):
      classKnn = KNeighborsClassifier(n_neighbors=1)
      classKnn.fit(X_train_by_class[src_class], y_train_by_class[src_class])
      knn_by_class[src_class] = classKnn

  #CBR adaptaion ML-CDH learning
  #pool for training pairs
  X_train_for_CDH = X_train
  y_train_for_CDH = y_train
  random_pairs_count = 10 * len(X_train_for_CDH)

  X_train_pairs_by_class = [[] for _ in range(num_classes)]
  y_train_pairs_by_class = [[] for _ in range(num_classes)]

  def addToPairList(source_y, X_train_pair, target_y):
    for src_class in range(num_classes):
      if source_y[src_class] == 1:
        X_train_pairs_by_class[src_class].append(X_train_pair)
        y_train_pairs_by_class[src_class].append(target_y)
        break
 
  #Assemble pairs
  if "1" in pair_selection:
    knn_for_CDH = KNeighborsClassifier(n_neighbors=2)
    knn_for_CDH.fit(X_train_for_CDH, y_train_for_CDH)
    X_train_neighbors_index = knn_for_CDH.kneighbors(X_train_for_CDH)[1]
    #we only select the second best neighbor, the first best neighbor of a case will be itself
    X_train_neighbors_index = X_train_neighbors_index[:,1]
    X_train_neighbors1 = X_train_for_CDH.iloc[X_train_neighbors_index]
    #IMPORTANT NOTE: left side of difference should be target, right side is source (or the retrieved case)
    y_train_neighbors = y_train_for_CDH.iloc[X_train_neighbors_index]

    for i in range(len(X_train_for_CDH.index)):
      addToPairList(y_train_for_CDH.iloc[i], pd.concat([X_train_for_CDH.iloc[i], X_train_neighbors1.iloc[i]],ignore_index=True), y_train_neighbors.iloc[i])
  if "2" in pair_selection:#not updated
    #2
    X_train_neighbors2_left = 0
    X_train_neighbors2_right = 0
    y_train_neighbors2_left = 0
    y_train_neighbors2_right = 0
    if pair_selection == 12:#not updated
      X_train_for_CDH_index = [*range(len(X_train_for_CDH))]
      np.random.shuffle(X_train_for_CDH_index) #shuffle in place
      X_train_neighbors2 = X_train_for_CDH[X_train_for_CDH_index]
      y_train_neighbors = y_train_for_CDH[X_train_for_CDH_index]
      X_train_differences2 = X_train_for_CDH - X_train_neighbors2
      y_train_differences2 = y_train_for_CDH - y_train_neighbors
  if "4" in pair_selection:
    left_index = np.random.choice(len(X_train_for_CDH), random_pairs_count)
    right_index = np.random.choice(len(X_train_for_CDH), random_pairs_count)
    X_train_neighbors2_left = X_train_for_CDH.iloc[left_index]
    y_train_neighbors2_left = y_train_for_CDH.iloc[left_index]
    X_train_neighbors2_right = X_train_for_CDH.iloc[right_index]
    y_train_neighbors2_right = y_train_for_CDH.iloc[right_index]
    for i in range(len(X_train_neighbors2_left.index)):
      addToPairList(y_train_neighbors2_left.iloc[i], pd.concat([X_train_neighbors2_left.iloc[i], X_train_neighbors2_right.iloc[i]],ignore_index=True),y_train_neighbors2_right.iloc[i])
  if "5" in pair_selection:#select C2C neighbors
    for src_class in range(num_classes):#src class
      for i in range(len(y_train_by_class[src_class])):
        X_train_neighbors5_left = (np.asarray(X_train_by_class[src_class][i])).reshape(1,-1)
        y_train_neighbors5_left = y_train_by_class[src_class][i]
        # y_train_neighbors5_left = np.asarray(y_train_by_class[src_class][i]).reshape(1, -1)

        for tgt_class in range(num_classes): #target classes
          if(tgt_class == src_class):#skip
            continue
          #.reshape(1, -1)
          X_test_retrieved_index = knn_by_class[tgt_class].kneighbors(X_train_neighbors5_left)[1]
          X_test_retrieved_index = X_test_retrieved_index[:,0][0]
          X_train_neighbors5_right = X_train_by_class[tgt_class][X_test_retrieved_index]
          y_train_neighbors5_right = y_train_by_class[tgt_class][X_test_retrieved_index]
          # print(X_train_neighbors5_left)
          X_train_neighbors5_left_series = pd.Series(X_train_neighbors5_left[0])
          # print(X_train_neighbors5_left)
          # print(X_train_neighbors5_right)
          
          addToPairList(y_train_neighbors5_left, 
                        pd.concat([X_train_neighbors5_left_series, X_train_neighbors5_right],ignore_index=True),y_train_neighbors5_right)

  
  #converting pairs data to nparray
  for i in range(num_classes):
    X_train_pairs_by_class[i] = np.asarray(X_train_pairs_by_class[i]) 
    y_train_pairs_by_class[i] = np.asarray(y_train_pairs_by_class[i])
  # print("test1vvfdi:")
  # print(X_train_pairs_by_class[0][0])
  # print("shape", X_train_pairs_by_class[0][0].shape)
  # print(y_train_pairs_by_class[0][0])

  #training adapt models
  adapt_models = [None for _ in range(num_classes)]
  for i in range(num_classes):
    adapt_model = None
    #select training pairs
    X_train_pairs = X_train_pairs_by_class[i]
    y_train_pairs = y_train_pairs_by_class[i]

    # print("Debug 389ngfv")
    # print(len(X_train_pairs))
    # print(len(X_train_pairs[0]))
    # print(len(y_train_pairs))
    # print("length is",len(X_train_pairs[0]))
    adapt_model = build_adaptation_model(X_train_pairs)
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=verbose_param)
    EPOCHS = 1000
    X_adapt_train, X_adapt_valid, y_adapt_train, y_adapt_valid = train_test_split(X_train_pairs, y_train_pairs, test_size=0.05, shuffle= True)
    early_history = adapt_model.fit(X_adapt_train, y_adapt_train, 
                        epochs=EPOCHS, validation_data=(X_adapt_valid, y_adapt_valid), verbose=verbose_param, 
                        callbacks=[early_stop, tfdocs.modeling.EpochDots()])

    print("adapt model last loss: ",early_history.history['val_loss'][-1])
    adapt_models[i] = adapt_model

  #normal CDH
  cdh_knns = [KNeighborsClassifier(n_neighbors=1) for _ in range(num_classes)]
  # cdh_knn = KNeighborsClassifier(n_neighbors=1)
  # might need multiple knn here , one for each class.
  for class_index in range(num_classes):
    X_train_pairs = X_train_pairs_by_class[class_index]
    y_train_pairs = y_train_pairs_by_class[class_index]
    cdh_knns[class_index].fit(X_train_pairs, y_train_pairs)

    
  return (dl_model, knn, adapt_models, cdh_knns, knn_by_class, X_train_by_class, y_train_by_class)

testing part


In [9]:
def test_models(X_train,y_train,X_test,y_test, EAC_adapt, dl_model, knn, adapt_models, cdh_knns, knn_by_class,X_train_by_class, y_train_by_class):
  y_test_pred = dl_model.predict(X_test)
  y_test_pred = tf.one_hot(tf.argmax(y_test_pred, 1), y_test_pred.shape[1])
  dl_loss = accuracy_score(y_test, y_test_pred)
  bal_dl_loss = balanced_accuracy_score(y_test.values.argmax(axis=1), tf.argmax(y_test_pred, 1))

  y_test_pred = knn.predict(X_test)  
  knn_loss = accuracy_score(y_test, y_test_pred)
  bal_knn_loss = balanced_accuracy_score(y_test.values.argmax(axis=1), np.argmax(y_test_pred,axis=1))

  #pure CBR retrieval (knn k=1), no need to retrain knn
  X_test_retrieved_index = knn.kneighbors(X_test, n_neighbors=1)[1]
  #we only select the best neighbor here!!!
  X_test_retrieved_index = X_test_retrieved_index[:,0]
  
  # print("debugging")
  # print(len(X_train))
  # print(max(X_test_retrieved_index))
  X_test_neighbors = X_train.iloc[X_test_retrieved_index]
  y_test_neighbors = y_train.iloc[X_test_retrieved_index]
  retrieval_loss = accuracy_score(y_test, y_test_neighbors)
  bal_retrieval_loss = balanced_accuracy_score(y_test.values.argmax(axis=1), y_test_neighbors.values.argmax(axis=1))

  #NN-CDH
  y_adapt_pred = []
  for i in range(len(X_test_neighbors.index)):
    for src_class in range(num_classes):
      if y_test_neighbors.iloc[i][src_class] == 1: #column label might be an issue. TOOD
        queryForAdapt = np.asarray([pd.concat([X_test_neighbors.iloc[i], X_test.iloc[i]],ignore_index=True)])
        y_pred = adapt_models[src_class].predict(queryForAdapt)
        y_adapt_pred.append(y_pred[0])
        continue
  y_adapt_pred = np.asarray(y_adapt_pred)
  y_adapt_pred = tf.one_hot(tf.argmax(y_adapt_pred, 1), y_adapt_pred.shape[1])
  retrieval_N_adapt_loss = accuracy_score(y_test, y_adapt_pred)
  bal_retrieval_N_adapt_loss = balanced_accuracy_score(y_test.values.argmax(axis=1), tf.argmax(y_adapt_pred, 1))
  
  
  #EAC + NN-CDH
  retrieval_N_EAC_adapt_loss = -1
  if("1" in EAC_adapt):
    #EAC retrieval
    y_adapt_pred = []
    X_test_EAC_retrieved_index = knn.kneighbors(X_test, n_neighbors=3)[1]
    for i in range(len(X_test_EAC_retrieved_index)):
      #for one query
      # voting = [0 for _ in range(num_classes)]
      voting = np.zeros((num_classes,), dtype=int)
      for j in range(len(X_test_EAC_retrieved_index[i])):
        retrieved_index = X_test_EAC_retrieved_index[i][j]
        for src_class in range(num_classes):
          if(y_train.iloc[retrieved_index][src_class] == 1):
            queryForAdapt = np.asarray([pd.concat([X_train.iloc[retrieved_index], X_test.iloc[i]],ignore_index=True)])
            y_pred = adapt_models[src_class].predict(queryForAdapt)
            voting = np.add(voting, y_pred[0])
      y_adapt_pred.append(voting)

    y_adapt_pred = np.asarray(y_adapt_pred)
    y_adapt_pred = tf.one_hot(tf.argmax(y_adapt_pred, 1), y_adapt_pred.shape[1])

    retrieval_N_EAC_adapt_loss = accuracy_score(y_test, y_adapt_pred)
    bal_retrieval_N_EAC_adapt_loss = balanced_accuracy_score(y_test.values.argmax(axis=1), tf.argmax(y_adapt_pred, 1))
  
  #C2C-EAC + NN-CDH
  C2C_EAC_NN_CDH_loss = -1
  bal_C2C_EAC_NN_CDH_loss = -1
  if("2" in EAC_adapt):
    y_adapt_pred = []
    #retrieval part
    X_test_EAC_retrieved_index_es = [[] for _ in range(num_classes)]
    for src_class in range(num_classes):
      X_test_EAC_retrieved_index_es[src_class] = knn_by_class[src_class].kneighbors(X_test, n_neighbors=1)[1]
      #we only select the best neighbor here!!!
      X_test_EAC_retrieved_index_es[src_class]  = X_test_EAC_retrieved_index_es[src_class][:,0]
    for i in range(len(X_test)):
      voting = np.zeros((num_classes,), dtype=int)
      for src_class in range(num_classes):
        retrieved_index = X_test_EAC_retrieved_index_es[src_class][i]
        #this index is relative to the class, use X_train_by_class to find it.
        retrieved_case = X_train_by_class[src_class][retrieved_index]
        retrieved_sol  = y_train_by_class[src_class][retrieved_index]
        queryForAdapt = np.asarray([pd.concat([retrieved_case, X_test.iloc[i]],ignore_index=True)])
        y_pred = adapt_models[src_class].predict(queryForAdapt)
        voting = np.add(voting, y_pred[0])
      y_adapt_pred.append(voting)
    
    y_adapt_pred = np.asarray(y_adapt_pred)
    y_adapt_pred = tf.one_hot(tf.argmax(y_adapt_pred, 1), y_adapt_pred.shape[1])

    C2C_EAC_NN_CDH_loss = accuracy_score(y_test, y_adapt_pred)
    bal_C2C_EAC_NN_CDH_loss = balanced_accuracy_score(y_test.values.argmax(axis=1), tf.argmax(y_adapt_pred, 1))
  #TODO

  #normal CDH
  y_diff_pred = []
  for i in range(len(X_test_neighbors.index)):
    for src_class in range(num_classes):
      if y_test_neighbors.iloc[i][src_class] == 1:
        queryForAdapt = np.asarray([pd.concat([X_test_neighbors.iloc[i], X_test.iloc[i]],ignore_index=True)])
        y_pred = cdh_knns[src_class].predict(queryForAdapt)
        y_diff_pred.append(y_pred[0])
        continue
  answer_pred_adapted =  y_diff_pred
  answer_pred_adapted = np.asarray(answer_pred_adapted)
  # print("normal Cdh",answer_pred_adapted)
  normal_cdh_loss = accuracy_score(y_test, answer_pred_adapted)
  bal_normal_cdh_loss = balanced_accuracy_score(y_test.values.argmax(axis=1), tf.argmax(answer_pred_adapted, 1))

  print("")
  print("dl_loss", dl_loss)
  print("knn_loss", knn_loss)
  print("retrieval_loss", retrieval_loss)
  print("retrieval_N_adapt_loss", retrieval_N_adapt_loss)
  print("CBR normal CDH loss", normal_cdh_loss)
  if("1" in EAC_adapt):
    print("retrieval_N_EAC_adapt_loss loss", retrieval_N_EAC_adapt_loss)
  if("2" in EAC_adapt):
    print("C2C_EAC_NN_CDH_loss ", C2C_EAC_NN_CDH_loss)
  return (dl_loss, knn_loss, retrieval_loss, retrieval_N_adapt_loss, normal_cdh_loss, retrieval_N_EAC_adapt_loss, C2C_EAC_NN_CDH_loss,
          bal_dl_loss, bal_knn_loss, bal_retrieval_loss, bal_retrieval_N_adapt_loss, bal_normal_cdh_loss, bal_retrieval_N_EAC_adapt_loss, bal_C2C_EAC_NN_CDH_loss)

## Special version for C2C rerun

The following version is for rerun after my mistake with C2C. Do not use in normal circumstances

In [10]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import mean_squared_error
# import tensorflow_docs as tfdocs
# import tensorflow_docs.plots
# import tensorflow_docs.modeling
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split

# #revamped feature config
# #EAC_adapt:0, no EAC; 1 EAC; 2 C2C-EAC
# #pair_selection: 1, neighbor; 4, random pairs; 5, C2C pairs
# #C2C IS ONLY USABLE WHEN EAC_adapt contains 2 and pair_selection contains 5 
# def train_models(X_train,y_train, EAC_adapt,pair_selection, pair_knowledge, verbose_param):
#   #pure DL model
#   dl_model = None

#   #pure knn
#   knn = KNeighborsClassifier(n_neighbors=3)
#   knn.fit(X_train, y_train)

#   #knn by class
#   X_train_by_class = [[] for _ in range(num_classes)]
#   y_train_by_class = [[] for _ in range(num_classes)]
#   knn_by_class= [0 for _ in range(num_classes) ]
#   if("2" in EAC_adapt and "5" in pair_selection):
#     for i in range(len(y_train.index)):
#       for src_class in range(num_classes):
#         if y_train.iloc[i][src_class] == 1:
#           X_train_by_class[src_class].append(X_train.iloc[i])
#           y_train_by_class[src_class].append(y_train.iloc[i])
#           break
#     for src_class in range(num_classes):
#       classKnn = KNeighborsClassifier(n_neighbors=1)
#       classKnn.fit(X_train_by_class[src_class], y_train_by_class[src_class])
#       knn_by_class[src_class] = classKnn

#   #CBR adaptaion ML-CDH learning
#   #pool for training pairs
#   X_train_for_CDH = X_train
#   y_train_for_CDH = y_train
#   random_pairs_count = 10 * len(X_train_for_CDH)

#   X_train_pairs_by_class = [[] for _ in range(num_classes)]
#   y_train_pairs_by_class = [[] for _ in range(num_classes)]

#   def addToPairList(source_y, X_train_pair, target_y):
#     for src_class in range(num_classes):
#       if source_y[src_class] == 1:
#         X_train_pairs_by_class[src_class].append(X_train_pair)
#         y_train_pairs_by_class[src_class].append(target_y)
#         break
 
#   #Assemble pairs
#   if "1" in pair_selection:
#     knn_for_CDH = KNeighborsClassifier(n_neighbors=2)
#     knn_for_CDH.fit(X_train_for_CDH, y_train_for_CDH)
#     X_train_neighbors_index = knn_for_CDH.kneighbors(X_train_for_CDH)[1]
#     #we only select the second best neighbor, the first best neighbor of a case will be itself
#     X_train_neighbors_index = X_train_neighbors_index[:,1]
#     X_train_neighbors1 = X_train_for_CDH.iloc[X_train_neighbors_index]
#     #IMPORTANT NOTE: left side of difference should be target, right side is source (or the retrieved case)
#     y_train_neighbors = y_train_for_CDH.iloc[X_train_neighbors_index]

#     for i in range(len(X_train_for_CDH.index)):
#       addToPairList(y_train_for_CDH.iloc[i], pd.concat([X_train_for_CDH.iloc[i], X_train_neighbors1.iloc[i]],ignore_index=True), y_train_neighbors.iloc[i])
#   if "2" in pair_selection:#not updated
#     #2
#     X_train_neighbors2_left = 0
#     X_train_neighbors2_right = 0
#     y_train_neighbors2_left = 0
#     y_train_neighbors2_right = 0
#     if pair_selection == 12:#not updated
#       X_train_for_CDH_index = [*range(len(X_train_for_CDH))]
#       np.random.shuffle(X_train_for_CDH_index) #shuffle in place
#       X_train_neighbors2 = X_train_for_CDH[X_train_for_CDH_index]
#       y_train_neighbors = y_train_for_CDH[X_train_for_CDH_index]
#       X_train_differences2 = X_train_for_CDH - X_train_neighbors2
#       y_train_differences2 = y_train_for_CDH - y_train_neighbors
#   if "4" in pair_selection:
#     left_index = np.random.choice(len(X_train_for_CDH), random_pairs_count)
#     right_index = np.random.choice(len(X_train_for_CDH), random_pairs_count)
#     X_train_neighbors2_left = X_train_for_CDH.iloc[left_index]
#     y_train_neighbors2_left = y_train_for_CDH.iloc[left_index]
#     X_train_neighbors2_right = X_train_for_CDH.iloc[right_index]
#     y_train_neighbors2_right = y_train_for_CDH.iloc[right_index]
#     for i in range(len(X_train_neighbors2_left.index)):
#       addToPairList(y_train_neighbors2_left.iloc[i], pd.concat([X_train_neighbors2_left.iloc[i], X_train_neighbors2_right.iloc[i]],ignore_index=True),y_train_neighbors2_right.iloc[i])
#   if "5" in pair_selection:#select C2C neighbors
#     for src_class in range(num_classes):#src class
#       for i in range(len(y_train_by_class[src_class])):
#         X_train_neighbors5_left = (np.asarray(X_train_by_class[src_class][i])).reshape(1,-1)
#         y_train_neighbors5_left = y_train_by_class[src_class][i]
#         # y_train_neighbors5_left = np.asarray(y_train_by_class[src_class][i]).reshape(1, -1)

#         for tgt_class in range(num_classes): #target classes
#           if(tgt_class == src_class):#skip
#             continue
#           #.reshape(1, -1)
#           X_test_retrieved_index = knn_by_class[tgt_class].kneighbors(X_train_neighbors5_left)[1]
#           X_test_retrieved_index = X_test_retrieved_index[:,0][0]
#           X_train_neighbors5_right = X_train_by_class[tgt_class][X_test_retrieved_index]
#           y_train_neighbors5_right = y_train_by_class[tgt_class][X_test_retrieved_index]
#           # print(X_train_neighbors5_left)
#           X_train_neighbors5_left_series = pd.Series(X_train_neighbors5_left[0])
#           # print(X_train_neighbors5_left)
#           # print(X_train_neighbors5_right)
          
#           addToPairList(y_train_neighbors5_left, 
#                         pd.concat([X_train_neighbors5_left_series, X_train_neighbors5_right],ignore_index=True),y_train_neighbors5_right)

  
#   #converting pairs data to nparray
#   for i in range(num_classes):
#     X_train_pairs_by_class[i] = np.asarray(X_train_pairs_by_class[i]) 
#     y_train_pairs_by_class[i] = np.asarray(y_train_pairs_by_class[i])
#   # print("test1vvfdi:")
#   # print(X_train_pairs_by_class[0][0])
#   # print("shape", X_train_pairs_by_class[0][0].shape)
#   # print(y_train_pairs_by_class[0][0])

#   #training adapt models
#   adapt_models = [None for _ in range(num_classes)]
#   for i in range(num_classes):
#     adapt_model = None
#     #select training pairs
#     X_train_pairs = X_train_pairs_by_class[i]
#     y_train_pairs = y_train_pairs_by_class[i]

#     # print("Debug 389ngfv")
#     # print(len(X_train_pairs))
#     # print(len(X_train_pairs[0]))
#     # print(len(y_train_pairs))
#     # print("length is",len(X_train_pairs[0]))
#     adapt_model = build_adaptation_model(X_train_pairs)
#     early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=verbose_param)
#     EPOCHS = 1000
#     X_adapt_train, X_adapt_valid, y_adapt_train, y_adapt_valid = train_test_split(X_train_pairs, y_train_pairs, test_size=0.05, shuffle= True)
#     early_history = adapt_model.fit(X_adapt_train, y_adapt_train, 
#                         epochs=EPOCHS, validation_data=(X_adapt_valid, y_adapt_valid), verbose=verbose_param, 
#                         callbacks=[early_stop, tfdocs.modeling.EpochDots()])

#     print("adapt model last loss: ",early_history.history['val_loss'][-1])
#     adapt_models[i] = adapt_model

#   #normal CDH
#   cdh_knns = None

    
#   return (dl_model, knn, adapt_models, cdh_knns, knn_by_class, X_train_by_class, y_train_by_class)

In [11]:
# def test_models(X_train,y_train,X_test,y_test, EAC_adapt, dl_model, knn, adapt_models, cdh_knns, knn_by_class,X_train_by_class, y_train_by_class):

#   dl_loss = 0
#   bal_dl_loss = 0

#   knn_loss = 0
#   bal_knn_loss = 0

#   #pure CBR retrieval (knn k=1), no need to retrain knn
#   # X_test_retrieved_index = knn.kneighbors(X_test, n_neighbors=1)[1]
#   # #we only select the best neighbor here!!!
#   # X_test_retrieved_index = X_test_retrieved_index[:,0]
  
#   # # print("debugging")
#   # # print(len(X_train))
#   # # print(max(X_test_retrieved_index))
#   # X_test_neighbors = X_train.iloc[X_test_retrieved_index]
#   # y_test_neighbors = y_train.iloc[X_test_retrieved_index]
#   retrieval_loss = 0
#   bal_retrieval_loss = 0

#   #NN-CDH
#   # y_adapt_pred = []
#   # for i in range(len(X_test_neighbors.index)):
#   #   for src_class in range(num_classes):
#   #     if y_test_neighbors.iloc[i][src_class] == 1: #column label might be an issue. TOOD
#   #       queryForAdapt = np.asarray([pd.concat([X_test_neighbors.iloc[i], X_test.iloc[i]],ignore_index=True)])
#   #       y_pred = adapt_models[src_class].predict(queryForAdapt)
#   #       y_adapt_pred.append(y_pred[0])
#   #       continue
#   # y_adapt_pred = np.asarray(y_adapt_pred)
#   # y_adapt_pred = tf.one_hot(tf.argmax(y_adapt_pred, 1), y_adapt_pred.shape[1])
#   retrieval_N_adapt_loss = 0# accuracy_score(y_test, y_adapt_pred)
#   bal_retrieval_N_adapt_loss =0# balanced_accuracy_score(y_test.values.argmax(axis=1), tf.argmax(y_adapt_pred, 1))
  
  
#   #EAC + NN-CDH
#   retrieval_N_EAC_adapt_loss = -1
#   if("1" in EAC_adapt):
#     # #EAC retrieval
#     # y_adapt_pred = []
#     # X_test_EAC_retrieved_index = knn.kneighbors(X_test, n_neighbors=3)[1]
#     # for i in range(len(X_test_EAC_retrieved_index)):
#     #   #for one query
#     #   # voting = [0 for _ in range(num_classes)]
#     #   voting = np.zeros((num_classes,), dtype=int)
#     #   for j in range(len(X_test_EAC_retrieved_index[i])):
#     #     retrieved_index = X_test_EAC_retrieved_index[i][j]
#     #     for src_class in range(num_classes):
#     #       if(y_train.iloc[retrieved_index][src_class] == 1):
#     #         queryForAdapt = np.asarray([pd.concat([X_train.iloc[retrieved_index], X_test.iloc[i]],ignore_index=True)])
#     #         y_pred = adapt_models[src_class].predict(queryForAdapt)
#     #         voting = np.add(voting, y_pred[0])
#     #   y_adapt_pred.append(voting)

#     # y_adapt_pred = np.asarray(y_adapt_pred)
#     # y_adapt_pred = tf.one_hot(tf.argmax(y_adapt_pred, 1), y_adapt_pred.shape[1])

#     retrieval_N_EAC_adapt_loss = 0 #accuracy_score(y_test, y_adapt_pred)
#     bal_retrieval_N_EAC_adapt_loss = 0# balanced_accuracy_score(y_test.values.argmax(axis=1), tf.argmax(y_adapt_pred, 1))
  
#   #C2C-EAC + NN-CDH
#   C2C_EAC_NN_CDH_loss = -1
#   bal_C2C_EAC_NN_CDH_loss = -1
#   if("2" in EAC_adapt):
#     y_adapt_pred = []
#     #retrieval part
#     X_test_EAC_retrieved_index_es = [[] for _ in range(num_classes)]
#     for src_class in range(num_classes):
#       X_test_EAC_retrieved_index_es[src_class] = knn_by_class[src_class].kneighbors(X_test, n_neighbors=1)[1]
#       #we only select the best neighbor here!!!
#       X_test_EAC_retrieved_index_es[src_class]  = X_test_EAC_retrieved_index_es[src_class][:,0]
#     for i in range(len(X_test)):
#       voting = np.zeros((num_classes,), dtype=int)
#       for src_class in range(num_classes):
#         retrieved_index = X_test_EAC_retrieved_index_es[src_class][i]
#         #this index is relative to the class, use X_train_by_class to find it.
#         retrieved_case = X_train_by_class[src_class][retrieved_index]
#         retrieved_sol  = y_train_by_class[src_class][retrieved_index]
#         queryForAdapt = np.asarray([pd.concat([retrieved_case, X_test.iloc[i]],ignore_index=True)])
#         y_pred = adapt_models[src_class].predict(queryForAdapt)
#         voting = np.add(voting, y_pred[0])
#       y_adapt_pred.append(voting)
    
#     y_adapt_pred = np.asarray(y_adapt_pred)
#     y_adapt_pred = tf.one_hot(tf.argmax(y_adapt_pred, 1), y_adapt_pred.shape[1])

#     C2C_EAC_NN_CDH_loss = accuracy_score(y_test, y_adapt_pred)
#     bal_C2C_EAC_NN_CDH_loss = balanced_accuracy_score(y_test.values.argmax(axis=1), tf.argmax(y_adapt_pred, 1))
#   #TODO

#   #normal CDH
#   # y_diff_pred = []
#   # for i in range(len(X_test_neighbors.index)):
#   #   for src_class in range(num_classes):
#   #     if y_test_neighbors.iloc[i][src_class] == 1:
#   #       queryForAdapt = np.asarray([pd.concat([X_test_neighbors.iloc[i], X_test.iloc[i]],ignore_index=True)])
#   #       y_pred = cdh_knns[src_class].predict(queryForAdapt)
#   #       y_diff_pred.append(y_pred[0])
#   #       continue
#   # answer_pred_adapted =  y_diff_pred
#   # answer_pred_adapted = np.asarray(answer_pred_adapted)
#   # # print("normal Cdh",answer_pred_adapted)
#   normal_cdh_loss = 0#accuracy_score(y_test, answer_pred_adapted)
#   bal_normal_cdh_loss =0# balanced_accuracy_score(y_test.values.argmax(axis=1), tf.argmax(answer_pred_adapted, 1))

#   print("")
#   print("dl_loss", dl_loss)
#   print("knn_loss", knn_loss)
#   print("retrieval_loss", retrieval_loss)
#   print("retrieval_N_adapt_loss", retrieval_N_adapt_loss)
#   print("CBR normal CDH loss", normal_cdh_loss)
#   if("1" in EAC_adapt):
#     print("retrieval_N_EAC_adapt_loss loss", retrieval_N_EAC_adapt_loss)
#   if("2" in EAC_adapt):
#     print("C2C_EAC_NN_CDH_loss ", C2C_EAC_NN_CDH_loss)
#   return (dl_loss, knn_loss, retrieval_loss, retrieval_N_adapt_loss, normal_cdh_loss, retrieval_N_EAC_adapt_loss, C2C_EAC_NN_CDH_loss,
#           bal_dl_loss, bal_knn_loss, bal_retrieval_loss, bal_retrieval_N_adapt_loss, bal_normal_cdh_loss, bal_retrieval_N_EAC_adapt_loss, bal_C2C_EAC_NN_CDH_loss)

# Graph and report

In [12]:
import seaborn as sns
def plotResults(arr_dl, arr_knn, arr_retrieval_loss, arr_retrieval_N_adapt_loss, arr_normal_cdh_loss):
  # kwargs = dict(alpha=0.5, bins=100)
  # plt.hist(arr_dl, **kwargs, color='g', label='dl')
  sns.distplot(arr_dl, hist = False, kde = True, color='g', label='NN',
                  kde_kws = {'linewidth': 3})
  # plt.hist(arr_knn, **kwargs, color='b', label='knn')
  sns.distplot(arr_knn, hist = False, kde = True, color='b', label='k-NN',
                  kde_kws = {'linewidth': 3})
  # plt.hist(arr_retrieval_loss, **kwargs, color='r', label='retrieval')
  sns.distplot(arr_retrieval_loss, hist = False, kde = True, color='r', label='retrieval',
                  kde_kws = {'linewidth': 3})
  # plt.hist(arr_retrieval_N_adapt_loss, **kwargs, color='c', label='retNadapt')
  sns.distplot(arr_retrieval_N_adapt_loss, hist = False, kde = True, color='c', label='CBR + DL-CDH',
                  kde_kws = {'linewidth': 3})
  sns.distplot(arr_normal_cdh_loss, hist = False, kde = True, color='y', label='CBR + normal CDH',
                  kde_kws = {'linewidth': 3})
  plt.gca().set(title='Density plot of loss', xlabel='accuracy')
  plt.xlim(0,1)
  plt.legend()

Maybe distribution plot is biased. Maybe we should show how often CBR beats DL instead.

In [13]:
def reportResults(X_test,arr_dl, arr_knn, arr_retrieval_loss, arr_retrieval_N_adapt_loss, arr_normal_cdh_loss):
  adapt_beats_DL = 0
  adapt_beats_DL_beats_ret = 0
  DL_beats_ret = 0
  testedRange = min(too_far, len(X_test))
  for i in range(testedRange):
    if arr_retrieval_N_adapt_loss[i] < arr_dl[i]:
      adapt_beats_DL += 1
      if arr_dl[i] < arr_retrieval_loss [i]:
        adapt_beats_DL_beats_ret += 1
    if arr_dl[i] < arr_retrieval_loss [i]:
      DL_beats_ret += 1
  print("num_cases_removed = ", num_cases_removed)
  print("total: ", testedRange)
  print("adapt_beats_DL: ", adapt_beats_DL)
  print("adapt_beats_DL_beats_ret: ", adapt_beats_DL_beats_ret)
  print("DL_beats_ret: ", DL_beats_ret)

  print("np.mean(arr_dl) = ", np.mean(arr_dl))
  print("np.mean(arr_knn) = ", np.mean(arr_knn))
  print("np.mean(arr_retrieval_loss) = ", np.mean(arr_retrieval_loss))
  print("np.mean(arr_retrieval_N_adapt_loss) = ", np.mean(arr_retrieval_N_adapt_loss))
  print("np.mean(arr_normal_cdh_loss) = ", np.mean(arr_normal_cdh_loss))


In [14]:
def reportResults_kFold(prompt,arr_dl, arr_knn, arr_retrieval_loss, arr_retrieval_N_adapt_loss, arr_normal_cdh_loss,retrieval_N_EAC_adapt_loss, C2C_EAC_NN_CDH_loss):
  print("reporting k Folds", prompt)
  print("np.mean(arr_dl) = ", np.mean(arr_dl))
  print("np.mean(arr_knn) = ", np.mean(arr_knn))
  print("np.mean(arr_retrieval_loss) = ", np.mean(arr_retrieval_loss))
  print("np.mean(arr_retrieval_N_adapt_loss) = ", np.mean(arr_retrieval_N_adapt_loss))
  print("np.mean(arr_normal_cdh_loss) = ", np.mean(arr_normal_cdh_loss))
  print("np.mean(retrieval_N_EAC_adapt_loss) = ", np.mean(retrieval_N_EAC_adapt_loss))
  print("np.mean(C2C_EAC_NN_CDH_loss) = ", np.mean(C2C_EAC_NN_CDH_loss))
  print("")
  print("reporting std", prompt)
  print("np.std(arr_dl) = ", np.std(arr_dl))
  print("np.std(arr_knn) = ", np.std(arr_knn))
  print("np.std(arr_retrieval_loss) = ", np.std(arr_retrieval_loss))
  print("np.std(arr_retrieval_N_adapt_loss) = ", np.std(arr_retrieval_N_adapt_loss))
  print("np.std(arr_normal_cdh_loss) = ", np.std(arr_normal_cdh_loss))
  print("np.std(retrieval_N_EAC_adapt_loss) = ", np.std(retrieval_N_EAC_adapt_loss))
  print("np.std(C2C_EAC_NN_CDH_loss) = ", np.std(C2C_EAC_NN_CDH_loss))

In [15]:
def reportGoodBad(X_test,arr_dl, arr_knn, arr_retrieval_loss, arr_retrieval_N_adapt_loss, arr_normal_cdh_loss):
  ret = []
  testedRange = min(too_far, len(X_test))
  for i in range(testedRange):
    winner = None
    if arr_dl[i] < arr_knn[i] and arr_dl[i] < arr_retrieval_loss[i] and arr_dl[i] < arr_retrieval_N_adapt_loss[i]:
      winner = "dl"
    elif arr_knn[i] < arr_dl[i] and arr_knn[i] < arr_retrieval_loss[i] and arr_knn[i] < arr_retrieval_N_adapt_loss[i]:
      winner = "knn"
    elif arr_retrieval_loss[i] < arr_dl[i] and arr_retrieval_loss[i] < arr_knn[i] and arr_retrieval_loss[i] < arr_retrieval_N_adapt_loss[i]:
      winner = "Retrieve"
    elif arr_retrieval_N_adapt_loss[i] < arr_dl[i] and arr_retrieval_N_adapt_loss[i] < arr_knn[i] and arr_retrieval_N_adapt_loss[i] < arr_retrieval_loss[i]:
      winner = "Adapt"
    print(X_test[i], winner)
    ret.append((X_test[i], winner))
  return ret

# Result: adapt model 2 + random/close pairs +extra pairs + partial. 


In [ ]:
X_train.shape

(587, 46)

In [ ]:
X_test.shape

(66, 46)

In [ ]:
#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = 1
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = 124
random_pairs_count = 100
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1
test_full_without_removal(EAC_adapt,pair_selection, pair_knowledge, 0)


Epoch: 0, accuracy:0.6593,  loss:0.6405,  val_accuracy:0.9091,  val_loss:0.4241,  
..............dl last loss:  0.31456121802330017

Epoch: 0, accuracy:0.6503,  loss:0.6393,  val_accuracy:0.5882,  val_loss:0.7942,  
...................adapt model last loss:  0.4176803231239319

Epoch: 0, accuracy:0.7536,  loss:0.5035,  val_accuracy:0.6316,  val_loss:0.7814,  
.................................adapt model last loss:  0.23807433247566223

dl_loss 0.8939393939393939
knn_loss 0.8484848484848485
retrieval_loss 0.7878787878787878
retrieval_N_adapt_loss 0.8787878787878788
CBR normal CDH loss 0.803030303030303
retrieval_N_EAC_adapt_loss loss 0.8787878787878788


(0.8939393939393939,
 0.8484848484848485,
 0.7878787878787878,
 0.8787878787878788,
 0.803030303030303,
 0.8787878787878788)

#Experiment after separation of training and testing

In [ ]:
#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from C2C neighbors, requires pair_selection "5"
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#4, rules from random pairs, with designated number of random pairs
#5, rules from C2C pairs
pair_selection = "145"
random_pairs_count = 100
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1
(dl_model, knn, adapt_models, cdh_knns, knn_by_class, X_train_by_class, y_train_by_class) = train_models(X_train,y_train,EAC_adapt,pair_selection, pair_knowledge, 0)
test_models(X_test,y_test, EAC_adapt, dl_model, knn, adapt_models, cdh_knns, knn_by_class, X_train_by_class, y_train_by_class)


Epoch: 0, accuracy:0.7257,  loss:0.5603,  val_accuracy:0.8333,  val_loss:0.4214,  
....................dl last loss:  0.42026832699775696

Epoch: 0, accuracy:0.5939,  loss:0.7038,  val_accuracy:0.7857,  val_loss:0.5692,  
..........................................adapt model last loss:  0.1687643676996231

Epoch: 0, accuracy:0.5739,  loss:0.6896,  val_accuracy:0.6389,  val_loss:0.5917,  
.................................adapt model last loss:  0.12306912988424301
debugging
587
561

dl_loss 0.8787878787878788
knn_loss 0.8333333333333334
retrieval_loss 0.8484848484848485
retrieval_N_adapt_loss 0.7727272727272727
CBR normal CDH loss 0.7121212121212122
retrieval_N_EAC_adapt_loss loss 0.8636363636363636
C2C_EAC_NN_CDH_loss  0.8636363636363636


(0.8787878787878788,
 0.8333333333333334,
 0.8484848484848485,
 0.7727272727272727,
 0.7121212121212122,
 0.8636363636363636,
 0.8636363636363636,
 0.8820276497695853,
 0.8373271889400922,
 0.8534562211981567,
 0.780184331797235,
 0.7119815668202765,
 0.8658986175115208,
 0.8677419354838709)

# KFOLD CODE

In [16]:
from sklearn.model_selection import KFold
def kFoldExperiment(X,y, num_of_runs):
  dl_loss_es=[]
  knn_loss_es = [] 
  retrieval_loss_es = []
  retrieval_N_adapt_loss_es = []
  normal_cdh_loss_es = []
  retrieval_N_EAC_adapt_loss_es = []
  C2C_EAC_NN_CDH_loss_es = []

  bal_dl_loss_es=[]
  bal_knn_loss_es = [] 
  bal_retrieval_loss_es = []
  bal_retrieval_N_adapt_loss_es = []
  bal_normal_cdh_loss_es = []
  bal_retrieval_N_EAC_adapt_loss_es = []
  bal_C2C_EAC_NN_CDH_loss_es = []

  for i in range(num_of_runs):
    kf = KFold(n_splits=10, shuffle=True)
    for train_index, test_index in kf.split(X):
        (X_train,X_test,y_train,y_test) = (None,None,None,None)
        (dl_model, knn, adapt_models, cdh_knns, knn_by_class, X_train_by_class, y_train_by_class) = (None,None,None,None,None,None,None)
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        (dl_model, knn, adapt_models, cdh_knns, knn_by_class, X_train_by_class, y_train_by_class) = train_models(X_train,y_train,EAC_adapt,pair_selection, pair_knowledge, 0)
        (dl_loss, knn_loss, retrieval_loss, retrieval_N_adapt_loss, normal_cdh_loss, retrieval_N_EAC_adapt_loss,C2C_EAC_NN_CDH_loss, 
        bal_dl_loss, bal_knn_loss, bal_retrieval_loss, bal_retrieval_N_adapt_loss, bal_normal_cdh_loss, bal_retrieval_N_EAC_adapt_loss, 
        bal_C2C_EAC_NN_CDH_loss) = test_models(X_train,y_train,X_test,y_test, EAC_adapt, dl_model, knn, adapt_models, cdh_knns, knn_by_class, X_train_by_class, y_train_by_class)
        
        dl_loss_es.append(dl_loss)
        knn_loss_es.append(knn_loss)
        retrieval_loss_es.append(retrieval_loss)
        retrieval_N_adapt_loss_es.append(retrieval_N_adapt_loss)
        normal_cdh_loss_es.append(normal_cdh_loss)
        retrieval_N_EAC_adapt_loss_es.append(retrieval_N_EAC_adapt_loss)
        C2C_EAC_NN_CDH_loss_es.append(C2C_EAC_NN_CDH_loss)

        bal_dl_loss_es.append(bal_dl_loss)
        bal_knn_loss_es.append(bal_knn_loss)
        bal_retrieval_loss_es.append(bal_retrieval_loss)
        bal_retrieval_N_adapt_loss_es.append(bal_retrieval_N_adapt_loss)
        bal_normal_cdh_loss_es.append(bal_normal_cdh_loss)
        bal_retrieval_N_EAC_adapt_loss_es.append(bal_retrieval_N_EAC_adapt_loss)
        bal_C2C_EAC_NN_CDH_loss_es.append(bal_C2C_EAC_NN_CDH_loss)

  print("")
  reportResults_kFold("average", dl_loss_es,knn_loss_es, retrieval_loss_es,retrieval_N_adapt_loss_es, normal_cdh_loss_es, retrieval_N_EAC_adapt_loss_es, C2C_EAC_NN_CDH_loss_es)
  print("")
  reportResults_kFold("balanced", bal_dl_loss_es, bal_knn_loss_es, bal_retrieval_loss_es, 
                      bal_retrieval_N_adapt_loss_es, bal_normal_cdh_loss_es, bal_retrieval_N_EAC_adapt_loss_es, bal_C2C_EAC_NN_CDH_loss_es)

In [17]:
# demonstrate data standardization with sklearn
from sklearn.preprocessing import StandardScaler
def scaleX(X, numericCols):
  # create scaler
  Xscaler = StandardScaler()
  scaled_features = X.copy()
  features = scaled_features[numericCols]
  # fit scaler on data
  Xscaler.fit(features.values)
  # apply transform
  features = Xscaler.transform(features.values)
  scaled_features[numericCols] = features
  return scaled_features

# X = scaleX(X,numericCols)

# Dataset loading: credit

In [18]:
filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/DL-CDH for classification/crx.data'
nominalCols = [0,3,4,5,6,8,9,11,12] 
numericCols = [1,2,7,10,13,14]
def GetDataMatrix():
    
    # Data frame with make and model
    X = pd.read_csv(filename,header=None, usecols=(0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,));

    y = pd.read_csv(filename,header=None, usecols=(15,));

    #remove rows with ? values
    rows_with_na = (X.values == '?').any(1)
    X = X[~rows_with_na]
    y = y[~rows_with_na]

    X, y = shuffle(X, y)
    print(X.head(0))
    # Turns categorical data into binary values across many columns
    #special one hot encoding with multiple values
    # market_category_dummies = X['Market Category'].str.get_dummies(sep=',')
    # X = pd.concat([X, market_category_dummies], axis=1)
    # X = X.drop(columns=['Market Category'])
    #normal one hot encoding
    X = pd.get_dummies(X, dummy_na = False, columns=nominalCols );
    y = pd.get_dummies(y, dummy_na=False)
    # Fill the null values with zeros
    # X.fillna(0, inplace=True);
    #there shouldn't be null, since it's already cleaned.
    print(X.isnull().sum())

    X = scaleX(X,numericCols)

    return (X, y)

##########

(X, y) = GetDataMatrix() #Gets the X,Y
num_classes = len(y.columns)

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
Index: []
1       0
2       0
7       0
10      0
13      0
14      0
0_a     0
0_b     0
3_l     0
3_u     0
3_y     0
4_g     0
4_gg    0
4_p     0
5_aa    0
5_c     0
5_cc    0
5_d     0
5_e     0
5_ff    0
5_i     0
5_j     0
5_k     0
5_m     0
5_q     0
5_r     0
5_w     0
5_x     0
6_bb    0
6_dd    0
6_ff    0
6_h     0
6_j     0
6_n     0
6_o     0
6_v     0
6_z     0
8_f     0
8_t     0
9_f     0
9_t     0
11_f    0
11_t    0
12_g    0
12_p    0
12_s    0
dtype: int64


In [33]:
X.head(5)

,1,2,7,10,13,14,0_a,0_b,3_l,3_u,3_y,4_g,4_gg,4_p,5_aa,5_c,5_cc,5_d,5_e,5_ff,5_i,5_j,5_k,5_m,5_q,5_r,5_w,5_x,6_bb,6_dd,6_ff,6_h,6_j,6_n,6_o,6_v,6_z,8_f,8_t,9_f,9_t,11_f,11_t,12_g,12_p,12_s
50,-0.486408,-0.861903,-0.406496,-0.504019,1.847202,-0.193125,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,1,0,0
614,0.577062,-0.082523,-0.629144,-0.504019,-0.121069,-0.193125,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0
210,0.661599,0.208127,2.302395,2.315895,1.300130,-0.193125,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,1,0,0
19,-1.042658,0.747620,-0.443604,0.905938,-0.501641,-0.193125,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,1,0,1,0,0
576,-0.112756,-0.861903,-0.146739,1.711628,-0.882213,-0.090253,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,1,0,1,0,0


In [ ]:
X.isna().sum()
nullseries =  X.isna().sum()
print(nullseries[nullseries > 0])

Series([], dtype: int64)


In [34]:
X.shape

(653, 46)

In [ ]:
y.shape

(653, 2)

In [ ]:
y.shape[0]

653

In [ ]:
X[1:10]

,1,2,7,10,13,14,0_a,0_b,3_l,3_u,3_y,4_g,4_gg,4_p,5_aa,5_c,5_cc,5_d,5_e,5_ff,5_i,5_j,5_k,5_m,5_q,5_r,5_w,5_x,6_bb,6_dd,6_ff,6_h,6_j,6_n,6_o,6_v,6_z,8_f,8_t,9_f,9_t,11_f,11_t,12_g,12_p,12_s
421,20.42,1.085,1.500,0,00108,7,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0
119,20.75,10.335,0.335,1,00080,50,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,1,0,0
347,26.75,4.500,2.500,0,00200,1210,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0
85,37.17,4.000,5.000,0,00280,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1
396,29.83,2.040,0.040,0,00128,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,0
368,23.58,11.500,3.000,0,00020,16,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1,0,0
463,36.58,0.290,0.000,10,00200,18,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0
109,19.75,0.750,0.795,5,00140,5,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,0
401,28.92,0.375,0.290,0,00220,140,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0


In [ ]:
y[1:10]

,15_+,15_-
421,0,1
119,1,0
347,0,1
85,0,1
396,0,1
368,0,1
463,0,1
109,0,1
401,0,1


maybe we can preprocess numerical values: normalization and standardization

Based on https://machinelearningmastery.com/how-to-improve-neural-network-stability-and-modeling-performance-with-data-scaling/

According to: https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/
They used StandardScaler.

IMPORTANT NOTE: Scaling is done on whole dataset instead of on train only. This might be a data leakage issue. However I intentionally do it for now because there is several outlier. If the outliers unluckily all fall in train or testing set, it is going to cause a problem, so I scale them together. 


In [ ]:
X.head(10)

,1,2,7,10,13,14,0_a,0_b,3_l,3_u,3_y,4_g,4_gg,4_p,5_aa,5_c,5_cc,5_d,5_e,5_ff,5_i,5_j,5_k,5_m,5_q,5_r,5_w,5_x,6_bb,6_dd,6_ff,6_h,6_j,6_n,6_o,6_v,6_z,8_f,8_t,9_f,9_t,11_f,11_t,12_g,12_p,12_s
525,45.17,1.500,2.500,0,00140,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,1,0,0
421,20.42,1.085,1.500,0,00108,7,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0
119,20.75,10.335,0.335,1,00080,50,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,1,1,0,0
347,26.75,4.500,2.500,0,00200,1210,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,0
85,37.17,4.000,5.000,0,00280,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1
396,29.83,2.040,0.040,0,00128,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,0
368,23.58,11.500,3.000,0,00020,16,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,1,1,0,0
463,36.58,0.290,0.000,10,00200,18,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0
109,19.75,0.750,0.795,5,00140,5,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,0
401,28.92,0.375,0.290,0,00220,140,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,0


In [ ]:
y.head()

,15_+,15_-
277,0,1
75,0,1
137,1,0
100,0,1
451,0,1


Split to train and test. random_state involved, so it might produce different results if using default value.

Not using cross validation here yet for simplicity. This is a todo for the future


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1, random_state= None)

#Trying 10 fold: Credit dataset

In [ ]:
X.shape

(653, 46)

In [ ]:
X_train.shape

NameError: ignored

In [19]:
#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from C2C neighbors, requires pair_selection "5"
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#4, rules from random pairs, with designated number of random pairs
#5, rules from C2C pairs
pair_selection = "145"
random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1
kFoldExperiment(X,y, 5)


Epoch: 0, accuracy:0.7178,  loss:0.5463,  val_accuracy:0.7119,  val_loss:0.5257,  
................dl last loss:  0.528183102607727

Epoch: 0, accuracy:0.7736,  loss:0.4716,  val_accuracy:0.8176,  val_loss:0.3972,  
.............................adapt model last loss:  0.22688037157058716

Epoch: 0, accuracy:0.8133,  loss:0.4324,  val_accuracy:0.8667,  val_loss:0.3849,  
.........................................adapt model last loss:  0.0984400063753128

dl_loss 0.8939393939393939
knn_loss 0.8939393939393939
retrieval_loss 0.8787878787878788
retrieval_N_adapt_loss 0.8333333333333334
CBR normal CDH loss 0.696969696969697
retrieval_N_EAC_adapt_loss loss 0.8484848484848485
C2C_EAC_NN_CDH_loss  0.8636363636363636

Epoch: 0, accuracy:0.6439,  loss:0.6380,  val_accuracy:0.8136,  val_loss:0.4855,  
.................dl last loss:  0.36039018630981445

Epoch: 0, accuracy:0.7554,  loss:0.4908,  val_accuracy:0.8038,  val_loss:0.4317,  
..........................adapt model last loss:  0.283227235



```
#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from C2C neighbors, requires pair_selection "5"
EAC_adapt = "1"
#1, rules from nearest pairs
#2, rules from random pairs
#4, rules from random pairs, with designated number of random pairs
#5, rules from C2C pairs
pair_selection = "14"
random_pairs_count = 100


reporting k Folds average
np.mean(arr_dl) =  0.8654079254079254
np.mean(arr_knn) =  0.8271328671328673
np.mean(arr_retrieval_loss) =  0.8009790209790209
np.mean(arr_retrieval_N_adapt_loss) =  0.8515617715617717
np.mean(arr_normal_cdh_loss) =  0.7811188811188812
np.mean(retrieval_N_EAC_adapt_loss) =  0.8562237762237762
np.mean(C2C_EAC_NN_CDH_loss) =  -1.0
reporting k Folds balanced
np.mean(arr_dl) =  0.8652446912304302
np.mean(arr_knn) =  0.8251316061542816
np.mean(arr_retrieval_loss) =  0.7974888101029917
np.mean(arr_retrieval_N_adapt_loss) =  0.8497658773919685
np.mean(arr_normal_cdh_loss) =  0.7792431371765137
np.mean(retrieval_N_EAC_adapt_loss) =  0.8541107267268713
np.mean(C2C_EAC_NN_CDH_loss) =  -1.0
```



It seems using C2C pairs make the performance of NN-CDH and EAC worse. probably because the number of neighbor/random pairs is less in proportion?



```
#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from C2C neighbors, requires pair_selection "5"
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#4, rules from random pairs, with designated number of random pairs
#5, rules from C2C pairs
pair_selection = "145"
random_pairs_count = 100
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

reporting k Folds average
np.mean(arr_dl) =  0.8807692307692309
np.mean(arr_knn) =  0.8271328671328673
np.mean(arr_retrieval_loss) =  0.8009790209790209
np.mean(arr_retrieval_N_adapt_loss) =  0.8102331002331002
np.mean(arr_normal_cdh_loss) =  0.7046153846153846
np.mean(retrieval_N_EAC_adapt_loss) =  0.8102564102564103
np.mean(C2C_EAC_NN_CDH_loss) =  0.8769230769230769
reporting k Folds balanced
np.mean(arr_dl) =  0.8810763834220481
np.mean(arr_knn) =  0.8251316061542816
np.mean(arr_retrieval_loss) =  0.7974888101029917
np.mean(arr_retrieval_N_adapt_loss) =  0.8098048810771484
np.mean(arr_normal_cdh_loss) =  0.70525270980614
np.mean(retrieval_N_EAC_adapt_loss) =  0.8111049480748365
np.mean(C2C_EAC_NN_CDH_loss) =  0.8795066413662238
```



Trying with many pairs 6000


```
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from C2C neighbors, requires pair_selection "5"
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#4, rules from random pairs, with designated number of random pairs
#5, rules from C2C pairs
pair_selection = "145"
random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

reporting k Folds average
np.mean(arr_dl) =  0.8683916083916084
np.mean(arr_knn) =  0.8407692307692308
np.mean(arr_retrieval_loss) =  0.7962937062937063
np.mean(arr_retrieval_N_adapt_loss) =  0.826946386946387
np.mean(arr_normal_cdh_loss) =  0.6877389277389278
np.mean(retrieval_N_EAC_adapt_loss) =  0.8207459207459207
np.mean(C2C_EAC_NN_CDH_loss) =  0.8769230769230769
reporting k Folds balanced
np.mean(arr_dl) =  0.8669290601811832
np.mean(arr_knn) =  0.8367214373123046
np.mean(arr_retrieval_loss) =  0.7914660473741275
np.mean(arr_retrieval_N_adapt_loss) =  0.8247365945924641
np.mean(arr_normal_cdh_loss) =  0.6845602809528976
np.mean(retrieval_N_EAC_adapt_loss) =  0.8197003771604517
np.mean(C2C_EAC_NN_CDH_loss) =  0.8765243902439024
```



Result with STD



```
reporting k Folds average
np.mean(arr_dl) =  0.8633193473193472
np.mean(arr_knn) =  0.8333892773892775
np.mean(arr_retrieval_loss) =  0.7978508158508157
np.mean(arr_retrieval_N_adapt_loss) =  0.822955710955711
np.mean(arr_normal_cdh_loss) =  0.7004755244755243
np.mean(retrieval_N_EAC_adapt_loss) =  0.8253659673659673
np.mean(C2C_EAC_NN_CDH_loss) =  0.8477342657342658

reporting std average
np.std(arr_dl) =  0.04812793953523209
np.std(arr_knn) =  0.05182127106440799
np.std(arr_retrieval_loss) =  0.04716352886173978
np.std(arr_retrieval_N_adapt_loss) =  0.04666709141480879
np.std(arr_normal_cdh_loss) =  0.055036124303965595
np.std(retrieval_N_EAC_adapt_loss) =  0.04961167474972114
np.std(C2C_EAC_NN_CDH_loss) =  0.046810015982655655

reporting k Folds balanced
np.mean(arr_dl) =  0.8616838402300354
np.mean(arr_knn) =  0.829758520464068
np.mean(arr_retrieval_loss) =  0.796232734866313
np.mean(arr_retrieval_N_adapt_loss) =  0.8204526389142416
np.mean(arr_normal_cdh_loss) =  0.6964849466074422
np.mean(retrieval_N_EAC_adapt_loss) =  0.8237734711606107
np.mean(C2C_EAC_NN_CDH_loss) =  0.8469433044164764

reporting std balanced
np.std(arr_dl) =  0.04654030431248819
np.std(arr_knn) =  0.053702612714060405
np.std(arr_retrieval_loss) =  0.047701179124887284
np.std(arr_retrieval_N_adapt_loss) =  0.046277433437263704
np.std(arr_normal_cdh_loss) =  0.05711400890760218
np.std(retrieval_N_EAC_adapt_loss) =  0.048432296047801665
np.std(C2C_EAC_NN_CDH_loss) =  0.04687153594730674
```



# Dataset loading: Balance

In [47]:
filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/DL-CDH for classification/balance-scale.data'
nominalCols = [] 
numericCols = [1,2,3,4]
def GetDataMatrix():
    
    # Data frame with make and model
    X = pd.read_csv(filename,header=None, usecols=(1,2,3,4,));

    y = pd.read_csv(filename,header=None, usecols=(0,));

    #remove rows with ? values
    if(X.values == '?'):
      rows_with_na = (X.values == '?').any(1)
      X = X[~rows_with_na]
      y = y[~rows_with_na]

    X, y = shuffle(X, y)
    print(X.head(0))
    # Turns categorical data into binary values across many columns
    #special one hot encoding with multiple values
    # market_category_dummies = X['Market Category'].str.get_dummies(sep=',')
    # X = pd.concat([X, market_category_dummies], axis=1)
    # X = X.drop(columns=['Market Category'])
    #normal one hot encoding
    X = pd.get_dummies(X, dummy_na = False, columns=nominalCols );
    y = pd.get_dummies(y, dummy_na=False)
    # Fill the null values with zeros
    # X.fillna(0, inplace=True);
    #there shouldn't be null, since it's already cleaned.
    print(X.isnull().sum())
    X = scaleX(X,numericCols)

    return (X, y)

##########

(X, y) = GetDataMatrix() #Gets the X,Y
num_classes = len(y.columns)

Empty DataFrame
Columns: [1, 2, 3, 4]
Index: []
1    0
2    0
3    0
4    0
dtype: int64


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if sys.path[0] == '':


In [48]:
X.shape

(625, 4)

In [18]:
X.head(5)

,1,2,3,4
447,0.707107,0.000000,1.414214,0.000000
419,0.707107,-0.707107,0.707107,1.414214
530,1.414214,-0.707107,-0.707107,-1.414214
297,0.000000,-0.707107,1.414214,0.000000
504,1.414214,-1.414214,-1.414214,1.414214


In [ ]:
y.head(5)

,0_B,0_L,0_R
147,0,0,1
366,0,1,0
42,0,0,1
99,0,0,1
228,0,1,0


In [ ]:
y.shape

(625, 3)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1, random_state= None)

# Trying 10 fold: Balance

In [49]:
from sklearn.model_selection import KFold

#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
# random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1
kFoldExperiment(X,y,5)
# kf = KFold(n_splits=10, shuffle=True, random_state=42)
# dl_loss_es=[]
# knn_loss_es = [] 
# retrieval_loss_es = []
# retrieval_N_adapt_loss_es = []
# normal_cdh_loss_es = []
# retrieval_N_EAC_adapt_loss_es = []

# bal_dl_loss_es=[]
# bal_knn_loss_es = [] 
# bal_retrieval_loss_es = []
# bal_retrieval_N_adapt_loss_es = []
# bal_normal_cdh_loss_es = []
# bal_retrieval_N_EAC_adapt_loss_es = []

# for train_index, test_index in kf.split(X):
#     # print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]

#     (dl_model, knn, adapt_models, cdh_knns, knn_by_class) = train_models(X_train,y_train,EAC_adapt,pair_selection, pair_knowledge, 0)
#     (dl_loss, knn_loss, retrieval_loss, retrieval_N_adapt_loss, normal_cdh_loss, retrieval_N_EAC_adapt_loss,
#      bal_dl_loss, bal_knn_loss, bal_retrieval_loss, bal_retrieval_N_adapt_loss, bal_normal_cdh_loss, bal_retrieval_N_EAC_adapt_loss) = test_models(X_test,y_test, EAC_adapt, dl_model, knn, adapt_models, cdh_knns, knn_by_class)
    
#     dl_loss_es.append(dl_loss)
#     knn_loss_es.append(knn_loss)
#     retrieval_loss_es.append(retrieval_loss)
#     retrieval_N_adapt_loss_es.append(retrieval_N_adapt_loss)
#     normal_cdh_loss_es.append(normal_cdh_loss)
#     retrieval_N_EAC_adapt_loss_es.append(retrieval_N_EAC_adapt_loss)

#     bal_dl_loss_es.append(bal_dl_loss)
#     bal_knn_loss_es.append(bal_knn_loss)
#     bal_retrieval_loss_es.append(bal_retrieval_loss)
#     bal_retrieval_N_adapt_loss_es.append(bal_retrieval_N_adapt_loss)
#     bal_normal_cdh_loss_es.append(bal_normal_cdh_loss)
#     bal_retrieval_N_EAC_adapt_loss_es.append(bal_retrieval_N_EAC_adapt_loss)
# reportResults_kFold("average", dl_loss_es,knn_loss_es, retrieval_loss_es,retrieval_N_adapt_loss_es, normal_cdh_loss_es, retrieval_N_EAC_adapt_loss_es)
# reportResults_kFold("balanced", bal_dl_loss_es, bal_knn_loss_es, bal_retrieval_loss_es, 
#                     bal_retrieval_N_adapt_loss_es, bal_normal_cdh_loss_es, bal_retrieval_N_EAC_adapt_loss_es)


Epoch: 0, accuracy:0.5537,  loss:1.0673,  val_accuracy:0.8966,  val_loss:0.5146,  
....................................................................................................
Epoch: 100, accuracy:1.0000,  loss:0.0028,  val_accuracy:1.0000,  val_loss:0.0015,  
....................................................................................................
Epoch: 200, accuracy:1.0000,  loss:0.0004,  val_accuracy:1.0000,  val_loss:0.0003,  
........................adapt model last loss:  0.00020665244664996862

Epoch: 0, accuracy:0.7760,  loss:0.5578,  val_accuracy:0.8667,  val_loss:0.3763,  
..................................................................................................adapt model last loss:  0.0008907954907044768

Epoch: 0, accuracy:0.7768,  loss:0.5421,  val_accuracy:0.8605,  val_loss:0.3324,  
..............................adapt model last loss:  0.03864926099777222

dl_loss 0
knn_loss 0
retrieval_loss 0
retrieval_N_adapt_loss 0
CBR normal CDH loss 0
r

In [23]:
np.std([1.0, 0.9206349206349206, 0.9841269841269841, 1.0, 0.9841269841269841, 0.967741935483871, 0.967741935483871, 0.9838709677419355, 0.9838709677419355, 1.0]
)

0.022544506902824456


```
#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = 1
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = 124
random_pairs_count = 100
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1
reporting k Folds average
np.mean(arr_dl) =  0.9729390681003585
np.mean(arr_knn) =  0.7889400921658986
np.mean(arr_retrieval_loss) =  0.7856118791602663
np.mean(arr_retrieval_N_adapt_loss) =  0.8959549411162314
np.mean(arr_normal_cdh_loss) =  0.7888888888888889
np.mean(retrieval_N_EAC_adapt_loss) =  0.9232206861239117

reporting k Folds balanced
np.mean(arr_dl) =  0.9469392463604986
np.mean(arr_knn) =  0.57911306672938
np.mean(arr_retrieval_loss) =  0.5713817031277086
np.mean(arr_retrieval_N_adapt_loss) =  0.774252877344564
np.mean(arr_normal_cdh_loss) =  0.6273414474125498
np.mean(retrieval_N_EAC_adapt_loss) =  0.8021367288833943
```


Using more pairs produces a lot better result

```
#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "1"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "124"
random_pairs_count = 6000

dl_loss 1.0
knn_loss 0.7580645161290323
retrieval_loss 0.7580645161290323
retrieval_N_adapt_loss 0.9838709677419355
CBR normal CDH loss 0.7258064516129032
retrieval_N_EAC_adapt_loss loss 0.9838709677419355
reporting k Folds average
np.mean(arr_dl) =  0.9776241679467486
np.mean(arr_knn) =  0.7968509984639016
np.mean(arr_retrieval_loss) =  0.7633640552995391
np.mean(arr_retrieval_N_adapt_loss) =  0.9760112647209421
np.mean(arr_normal_cdh_loss) =  0.7808499743983615
np.mean(retrieval_N_EAC_adapt_loss) =  0.9759856630824373
np.mean(C2C_EAC_NN_CDH_loss) =  -1.0
reporting k Folds balanced
np.mean(arr_dl) =  0.9546992387101767
np.mean(arr_knn) =  0.5762784229508773
np.mean(arr_retrieval_loss) =  0.5512892255629526
np.mean(arr_retrieval_N_adapt_loss) =  0.9744648511841227
np.mean(arr_normal_cdh_loss) =  0.6132406730544281
np.mean(retrieval_N_EAC_adapt_loss) =  0.9588398511841227
np.mean(C2C_EAC_NN_CDH_loss) =  -1.0
```





```
#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

reporting k Folds average
np.mean(arr_dl) =  0.9712237583205325
np.mean(arr_knn) =  0.7870967741935484
np.mean(arr_retrieval_loss) =  0.7790322580645161
np.mean(arr_retrieval_N_adapt_loss) =  0.969636456733231
np.mean(arr_normal_cdh_loss) =  0.6608550947260625
np.mean(retrieval_N_EAC_adapt_loss) =  0.9744239631336405
np.mean(C2C_EAC_NN_CDH_loss) =  0.967741935483871
reporting k Folds balanced
np.mean(arr_dl) =  0.937722595627719
np.mean(arr_knn) =  0.5676574856948584
np.mean(arr_retrieval_loss) =  0.5616761381193102
np.mean(arr_retrieval_N_adapt_loss) =  0.9574465246258415
np.mean(arr_normal_cdh_loss) =  0.5422665812453158
np.mean(retrieval_N_EAC_adapt_loss) =  0.9662637289269167
np.mean(C2C_EAC_NN_CDH_loss) =  0.9765873015873017
```



Using STD with multi runs



```
reporting k Folds average
np.mean(arr_dl) =  0.9737532002048133
np.mean(arr_knn) =  0.7964925755248334
np.mean(arr_retrieval_loss) =  0.7769943676395289
np.mean(arr_retrieval_N_adapt_loss) =  0.9763184843830007
np.mean(arr_normal_cdh_loss) =  0.6561546338965694
np.mean(retrieval_N_EAC_adapt_loss) =  0.9737685611879161
np.mean(C2C_EAC_NN_CDH_loss) =  0.9516129032258065

reporting std average
np.std(arr_dl) =  0.01971400906942525
np.std(arr_knn) =  0.051138111374122844
np.std(arr_retrieval_loss) =  0.05455483683696814
np.std(arr_retrieval_N_adapt_loss) =  0.017609107718912546
np.std(arr_normal_cdh_loss) =  0.05414928499979276
np.std(retrieval_N_EAC_adapt_loss) =  0.01859286877318543
np.std(C2C_EAC_NN_CDH_loss) =  0.0

reporting k Folds balanced
np.mean(arr_dl) =  0.9591936439458675
np.mean(arr_knn) =  0.57784063014861
np.mean(arr_retrieval_loss) =  0.5630270785334525
np.mean(arr_retrieval_N_adapt_loss) =  0.9663559535837376
np.mean(arr_normal_cdh_loss) =  0.5497414496609297
np.mean(retrieval_N_EAC_adapt_loss) =  0.9639199490045327
np.mean(C2C_EAC_NN_CDH_loss) =  0.9043478260869565

reporting std balanced
np.std(arr_dl) =  0.04885578713837359
np.std(arr_knn) =  0.026486130761340414
np.std(arr_retrieval_loss) =  0.02865470426646368
np.std(arr_retrieval_N_adapt_loss) =  0.045669393639572925
np.std(arr_normal_cdh_loss) =  0.08541824772469106
np.std(retrieval_N_EAC_adapt_loss) =  0.046519168993125026
np.std(C2C_EAC_NN_CDH_loss) =  0.0
```



# Dataset: Car

https://archive.ics.uci.edu/ml/datasets/car+evaluation

In [50]:
filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/DL-CDH for classification/car.data'
nominalCols = [0,1,2,3,4,5] 
numericCols = []
def GetDataMatrix():
    
    # Data frame with make and model
    X = pd.read_csv(filename,header=None, usecols=(0,1,2,3,4,5,));

    y = pd.read_csv(filename,header=None, usecols=(6,));

    #no missing values
    #remove rows with ? values
    # if(X.values == '?'):
    #   rows_with_na = (X.values == '?').any(1)
    #   X = X[~rows_with_na]
    #   y = y[~rows_with_na]

    X, y = shuffle(X, y)
    print(X.head(0))
    # Turns categorical data into binary values across many columns
    #special one hot encoding with multiple values
    # market_category_dummies = X['Market Category'].str.get_dummies(sep=',')
    # X = pd.concat([X, market_category_dummies], axis=1)
    # X = X.drop(columns=['Market Category'])
    #normal one hot encoding
    X = pd.get_dummies(X, dummy_na = False, columns=nominalCols );
    y = pd.get_dummies(y, dummy_na=False)
    # Fill the null values with zeros
    # X.fillna(0, inplace=True);
    #there shouldn't be null, since it's already cleaned.
    print(X.isnull().sum())
    
    # X = scaleX(X,numericCols)

    return (X, y)

##########

(X, y) = GetDataMatrix() #Gets the X,Y
num_classes = len(y.columns)

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5]
Index: []
0_high     0
0_low      0
0_med      0
0_vhigh    0
1_high     0
1_low      0
1_med      0
1_vhigh    0
2_2        0
2_3        0
2_4        0
2_5more    0
3_2        0
3_4        0
3_more     0
4_big      0
4_med      0
4_small    0
5_high     0
5_low      0
5_med      0
dtype: int64


In [ ]:
X.shape

(1728, 21)

# Trying 10 fold: Car

In [51]:
from sklearn.model_selection import KFold

#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
# random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

kFoldExperiment(X,y,5)

# kf = KFold(n_splits=10, shuffle=True, random_state=42)
# dl_loss_es=[]
# knn_loss_es = [] 
# retrieval_loss_es = []
# retrieval_N_adapt_loss_es = []
# normal_cdh_loss_es = []
# retrieval_N_EAC_adapt_loss_es = []

# bal_dl_loss_es=[]
# bal_knn_loss_es = [] 
# bal_retrieval_loss_es = []
# bal_retrieval_N_adapt_loss_es = []
# bal_normal_cdh_loss_es = []
# bal_retrieval_N_EAC_adapt_loss_es = []

# for train_index, test_index in kf.split(X):
#     # print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]

#     (dl_model, knn, adapt_models, cdh_knns, knn_by_class) = train_models(X_train,y_train,EAC_adapt,pair_selection, pair_knowledge, 0)
#     (dl_loss, knn_loss, retrieval_loss, retrieval_N_adapt_loss, normal_cdh_loss, retrieval_N_EAC_adapt_loss,
#      bal_dl_loss, bal_knn_loss, bal_retrieval_loss, bal_retrieval_N_adapt_loss, bal_normal_cdh_loss, bal_retrieval_N_EAC_adapt_loss) = test_models(X_test,y_test, EAC_adapt, dl_model, knn, adapt_models, cdh_knns, knn_by_class)
    
#     dl_loss_es.append(dl_loss)
#     knn_loss_es.append(knn_loss)
#     retrieval_loss_es.append(retrieval_loss)
#     retrieval_N_adapt_loss_es.append(retrieval_N_adapt_loss)
#     normal_cdh_loss_es.append(normal_cdh_loss)
#     retrieval_N_EAC_adapt_loss_es.append(retrieval_N_EAC_adapt_loss)

#     bal_dl_loss_es.append(bal_dl_loss)
#     bal_knn_loss_es.append(bal_knn_loss)
#     bal_retrieval_loss_es.append(bal_retrieval_loss)
#     bal_retrieval_N_adapt_loss_es.append(bal_retrieval_N_adapt_loss)
#     bal_normal_cdh_loss_es.append(bal_normal_cdh_loss)
#     bal_retrieval_N_EAC_adapt_loss_es.append(bal_retrieval_N_EAC_adapt_loss)
# reportResults_kFold("average", dl_loss_es,knn_loss_es, retrieval_loss_es,retrieval_N_adapt_loss_es, normal_cdh_loss_es, retrieval_N_EAC_adapt_loss_es)
# reportResults_kFold("balanced", bal_dl_loss_es, bal_knn_loss_es, bal_retrieval_loss_es, 
#                     bal_retrieval_N_adapt_loss_es, bal_normal_cdh_loss_es, bal_retrieval_N_EAC_adapt_loss_es)


Epoch: 0, accuracy:0.7196,  loss:0.7045,  val_accuracy:0.9008,  val_loss:0.3408,  
....................................................................................................adapt model last loss:  0.00046726063010282815

Epoch: 0, accuracy:0.5416,  loss:1.1052,  val_accuracy:0.7273,  val_loss:0.8457,  
.......................................adapt model last loss:  0.07943802326917648

Epoch: 0, accuracy:0.8790,  loss:0.3335,  val_accuracy:0.9712,  val_loss:0.0971,  
.............................................................................adapt model last loss:  3.989213936961278e-08

Epoch: 0, accuracy:0.5110,  loss:1.1533,  val_accuracy:0.6154,  val_loss:0.9755,  
....................................................................................................
Epoch: 100, accuracy:1.0000,  loss:0.0006,  val_accuracy:1.0000,  val_loss:0.0263,  
....................................................................................................
Epoch: 200, accuracy:1.0



```
#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = 1
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = 124
random_pairs_count = 100
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

reporting k Folds average
np.mean(arr_dl) =  1.0
np.mean(arr_knn) =  0.8281119774163195
np.mean(arr_retrieval_loss) =  0.8003394273423847
np.mean(arr_retrieval_N_adapt_loss) =  0.8790361607742977
np.mean(arr_normal_cdh_loss) =  0.7297284581260921
np.mean(retrieval_N_EAC_adapt_loss) =  0.9074001881973384

reporting k Folds balanced
np.mean(arr_dl) =  1.0
np.mean(arr_knn) =  0.6022642862673279
np.mean(arr_retrieval_loss) =  0.6162098982135521
np.mean(arr_retrieval_N_adapt_loss) =  0.6585996871178228
np.mean(arr_normal_cdh_loss) =  0.4869124047923922
np.mean(retrieval_N_EAC_adapt_loss) =  0.6974682316945179
```


IMPORTANT NOTE:
Interestingly we get bad accuracy scores here. There are 2 ways to improve
1. more pairs for NN-CDH-EAC  because they only have 100 random pairs per net
EAC style pairs also
C2C-EAC style pairs also
C2C-EAC classification process
2. Using the retrieval of the original EAC. Notice that the knn is performing poorly.  This is a lot worse than the special knn in Vahid's paper.



```
#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
random_pairs_count = 30000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

reporting k Folds average
np.mean(arr_dl) =  0.999421965317919
np.mean(arr_knn) =  0.8275238607339697
np.mean(arr_retrieval_loss) =  0.7812407581664202
np.mean(arr_retrieval_N_adapt_loss) =  0.9959470358919209
np.mean(arr_normal_cdh_loss) =  0.7019794327194515
np.mean(retrieval_N_EAC_adapt_loss) =  0.9965250705740019
np.mean(C2C_EAC_NN_CDH_loss) =  0.9941860465116279
reporting k Folds balanced
np.mean(arr_dl) =  0.9997826086956522
np.mean(arr_knn) =  0.6089479409264086
np.mean(arr_retrieval_loss) =  0.5769741353708231
np.mean(arr_retrieval_N_adapt_loss) =  0.9857708565589001
np.mean(arr_normal_cdh_loss) =  0.5704297387587864
np.mean(retrieval_N_EAC_adapt_loss) =  0.9849960233297986
np.mean(C2C_EAC_NN_CDH_loss) =  0.99375
```



Down scale to 6000 pairs



```
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

reporting k Folds average
np.mean(arr_dl) =  0.9965284312407581
np.mean(arr_knn) =  0.8275238607339697
np.mean(arr_retrieval_loss) =  0.7812407581664202
np.mean(arr_retrieval_N_adapt_loss) =  0.9837814222341711
np.mean(arr_normal_cdh_loss) =  0.6776986154052964
np.mean(retrieval_N_EAC_adapt_loss) =  0.991897432450598
np.mean(C2C_EAC_NN_CDH_loss) =  0.9883720930232558

reporting k Folds balanced
np.mean(arr_dl) =  0.987081248262441
np.mean(arr_knn) =  0.6089479409264086
np.mean(arr_retrieval_loss) =  0.5769741353708231
np.mean(arr_retrieval_N_adapt_loss) =  0.9508979312902591
np.mean(arr_normal_cdh_loss) =  0.5524140549287838
np.mean(retrieval_N_EAC_adapt_loss) =  0.9698078170915923
np.mean(C2C_EAC_NN_CDH_loss) =  0.9917174796747967
```



Using 10 * training number as random pair number

```
reporting k Folds average
np.mean(arr_dl) =  0.9965250705740019
np.mean(arr_knn) =  0.823440650625084
np.mean(arr_retrieval_loss) =  0.7760182820271542
np.mean(arr_retrieval_N_adapt_loss) =  0.9930535018147599
np.mean(arr_normal_cdh_loss) =  0.6753461486758974
np.mean(retrieval_N_EAC_adapt_loss) =  0.9918974324505981
np.mean(C2C_EAC_NN_CDH_loss) =  0.9941860465116279

reporting k Folds balanced
np.mean(arr_dl) =  0.9984232199181632
np.mean(arr_knn) =  0.6108833970029404
np.mean(arr_retrieval_loss) =  0.5309070704038723
np.mean(arr_retrieval_N_adapt_loss) =  0.9715752359502361
np.mean(arr_normal_cdh_loss) =  0.5354995320623634
np.mean(retrieval_N_EAC_adapt_loss) =  0.9572234387059968
np.mean(C2C_EAC_NN_CDH_loss) =  0.9944444444444445
```



with STD



```
reporting k Folds average
np.mean(arr_dl) =  0.9974559752654926
np.mean(arr_knn) =  0.8313624143029977
np.mean(arr_retrieval_loss) =  0.7810202984272078
np.mean(arr_retrieval_N_adapt_loss) =  0.9890065869068421
np.mean(arr_normal_cdh_loss) =  0.6822832369942198
np.mean(retrieval_N_EAC_adapt_loss) =  0.9936355692969485
np.mean(C2C_EAC_NN_CDH_loss) =  0.9941860465116279

reporting std average
np.std(arr_dl) =  0.005191159835236904
np.std(arr_knn) =  0.02526178528859611
np.std(arr_retrieval_loss) =  0.028800644274238243
np.std(arr_retrieval_N_adapt_loss) =  0.008749778786491984
np.std(arr_normal_cdh_loss) =  0.03448260562534618
np.std(retrieval_N_EAC_adapt_loss) =  0.007427678396361223
np.std(C2C_EAC_NN_CDH_loss) =  0.0

reporting k Folds balanced
np.mean(arr_dl) =  0.9974142678849565
np.mean(arr_knn) =  0.6056267519903155
np.mean(arr_retrieval_loss) =  0.5623310611064158
np.mean(arr_retrieval_N_adapt_loss) =  0.9553387342412119
np.mean(arr_normal_cdh_loss) =  0.5360104748568112
np.mean(retrieval_N_EAC_adapt_loss) =  0.9789716923497828
np.mean(C2C_EAC_NN_CDH_loss) =  0.9980314960629921

reporting std balanced
np.std(arr_dl) =  0.006836961784849787
np.std(arr_knn) =  0.06597657617907499
np.std(arr_retrieval_loss) =  0.07878962568591576
np.std(arr_retrieval_N_adapt_loss) =  0.03835590364768343
np.std(arr_normal_cdh_loss) =  0.0784487264211488
np.std(retrieval_N_EAC_adapt_loss) =  0.028302792154520584
np.std(C2C_EAC_NN_CDH_loss) =  0.0
```



# Dataset: Yeast

Starting from here, the datasets are chosen based on this paper

Anup-to-datecomparisonofstate-of-the-artclassificationalgorithms.pdf

In [20]:
filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/DL-CDH for classification/yeast.data'
nominalCols = [] 
numericCols = [1,2,3,4,5,6,7,8]
def GetDataMatrix():
    
    # Data frame with make and model
    X = pd.read_csv(filename,header=None, usecols=(1,2,3,4,5,6,7,8,),delim_whitespace=True);

    y = pd.read_csv(filename,header=None, usecols=(9,),delim_whitespace=True);

    #no missing values
    #remove rows with ? values
    # if(X.values == '?'):
    #   rows_with_na = (X.values == '?').any(1)
    #   X = X[~rows_with_na]
    #   y = y[~rows_with_na]

    X, y = shuffle(X, y)
    print(X.head(0))
    # Turns categorical data into binary values across many columns
    #special one hot encoding with multiple values
    # market_category_dummies = X['Market Category'].str.get_dummies(sep=',')
    # X = pd.concat([X, market_category_dummies], axis=1)
    # X = X.drop(columns=['Market Category'])
    #normal one hot encoding
    X = pd.get_dummies(X, dummy_na = False, columns=nominalCols );
    y = pd.get_dummies(y, dummy_na=False)
    # Fill the null values with zeros
    # X.fillna(0, inplace=True);
    #there shouldn't be null, since it's already cleaned.
    print(X.isnull().sum())
    X = scaleX(X,numericCols)

    return (X, y)

##########

(X, y) = GetDataMatrix() #Gets the X,Y
num_classes = len(y.columns)

Empty DataFrame
Columns: [1, 2, 3, 4, 5, 6, 7, 8]
Index: []
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
dtype: int64


In [ ]:
X.head(10)

,1,2,3,4,5,6,7,8
312,0.581981,0.807760,0.115030,-0.154584,-0.09759,-0.099131,0.521219,-0.527919
138,-1.530905,-1.452445,0.922961,-0.811272,-0.09759,-0.099131,0.001983,0.223574
942,0.509123,-0.080178,1.269217,-0.008654,-0.09759,-0.099131,0.348141,-0.527919
1318,-0.583749,-0.322342,-0.692901,-0.665341,-0.09759,-0.099131,0.175062,1.350815
224,0.436265,-0.725950,0.922961,-0.373480,-0.09759,-0.099131,-1.382649,-0.527919
266,-0.510891,-0.887394,0.461286,-0.884237,-0.09759,-0.099131,-0.517254,-0.527919
227,-1.166614,-1.129558,0.461286,-1.103133,-0.09759,-0.099131,-2.421122,-0.621856
639,0.071974,0.807760,0.115030,-0.008654,-0.09759,-0.099131,0.694298,-0.527919
1438,-0.073742,0.404152,-1.731670,-0.227549,-0.09759,-0.099131,1.213535,-0.527919
29,2.184860,3.067965,-1.616251,0.939895,-0.09759,-0.099131,1.040456,0.505385


In [ ]:
y.head(5)

,9_CYT,9_ERL,9_EXC,9_ME1,9_ME2,9_ME3,9_MIT,9_NUC,9_POX,9_VAC
312,1,0,0,0,0,0,0,0,0,0
138,1,0,0,0,0,0,0,0,0,0
942,0,0,0,0,0,0,0,1,0,0
1318,1,0,0,0,0,0,0,0,0,0
224,1,0,0,0,0,0,0,0,0,0


#Trying 10 fold: Yeast

In [21]:
from sklearn.model_selection import KFold

#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
# random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

kFoldExperiment(X,y,5)


Epoch: 0, accuracy:0.4555,  loss:1.6689,  val_accuracy:0.5299,  val_loss:1.3144,  
....................dl last loss:  1.2104250192642212

Epoch: 0, accuracy:0.4275,  loss:1.5861,  val_accuracy:0.4818,  val_loss:1.3922,  
.............................................................................adapt model last loss:  0.5644859671592712

Epoch: 0, accuracy:0.0323,  loss:3.5654,  val_accuracy:0.0000,  val_loss:3.4576,  
...........................................adapt model last loss:  1.7045118808746338

Epoch: 0, accuracy:0.2130,  loss:2.2051,  val_accuracy:0.3529,  val_loss:1.9002,  
.....................................adapt model last loss:  0.9835273623466492

Epoch: 0, accuracy:0.2529,  loss:2.0611,  val_accuracy:0.4146,  val_loss:1.5452,  
...............................................adapt model last loss:  1.29465651512146

Epoch: 0, accuracy:0.2395,  loss:2.1240,  val_accuracy:0.4255,  val_loss:1.5765,  
.....................................................adapt model las

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')



dl_loss 0.5100671140939598
knn_loss 0.38926174496644295
retrieval_loss 0.4697986577181208
retrieval_N_adapt_loss 0.4429530201342282
CBR normal CDH loss 0.38926174496644295
retrieval_N_EAC_adapt_loss loss 0.5100671140939598
C2C_EAC_NN_CDH_loss  0.5033557046979866

Epoch: 0, accuracy:0.4567,  loss:1.7974,  val_accuracy:0.5224,  val_loss:1.3108,  
..............dl last loss:  1.1410129070281982

Epoch: 0, accuracy:0.4358,  loss:1.5426,  val_accuracy:0.4800,  val_loss:1.4162,  
........................................................adapt model last loss:  0.5898144841194153

Epoch: 0, accuracy:0.0000,  loss:5.9844,  val_accuracy:0.0000,  val_loss:4.9557,  
..................................adapt model last loss:  2.2318098545074463

Epoch: 0, accuracy:0.2268,  loss:2.2229,  val_accuracy:0.3529,  val_loss:1.6281,  
..............................adapt model last loss:  0.9233530163764954

Epoch: 0, accuracy:0.2889,  loss:2.0823,  val_accuracy:0.2308,  val_loss:2.0950,  
...................

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')



dl_loss 0.6418918918918919
knn_loss 0.4527027027027027
retrieval_loss 0.5472972972972973
retrieval_N_adapt_loss 0.47297297297297297
CBR normal CDH loss 0.39864864864864863
retrieval_N_EAC_adapt_loss loss 0.5067567567567568
C2C_EAC_NN_CDH_loss  0.6148648648648649

Epoch: 0, accuracy:0.4101,  loss:1.7917,  val_accuracy:0.5299,  val_loss:1.4002,  
.....................dl last loss:  1.139782428741455

Epoch: 0, accuracy:0.4237,  loss:1.5812,  val_accuracy:0.5036,  val_loss:1.3592,  
..........................................................................adapt model last loss:  0.4989911913871765

Epoch: 0, accuracy:0.1122,  loss:6.8043,  val_accuracy:0.0000,  val_loss:3.2882,  
........................................adapt model last loss:  1.7471212148666382

Epoch: 0, accuracy:0.2003,  loss:2.3299,  val_accuracy:0.4545,  val_loss:1.5595,  
...........................................................adapt model last loss:  0.912898063659668

Epoch: 0, accuracy:0.2396,  loss:2.1237,  va

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')



dl_loss 0.6148648648648649
knn_loss 0.5
retrieval_loss 0.4864864864864865
retrieval_N_adapt_loss 0.49324324324324326
CBR normal CDH loss 0.41216216216216217
retrieval_N_EAC_adapt_loss loss 0.5608108108108109
C2C_EAC_NN_CDH_loss  0.5608108108108109

Epoch: 0, accuracy:0.3993,  loss:1.7825,  val_accuracy:0.4776,  val_loss:1.4689,  
..................dl last loss:  1.255313515663147

Epoch: 0, accuracy:0.4217,  loss:1.5791,  val_accuracy:0.5235,  val_loss:1.3203,  
.........................................................................adapt model last loss:  0.5613541007041931

Epoch: 0, accuracy:0.1449,  loss:3.7984,  val_accuracy:0.2500,  val_loss:3.2786,  
............................adapt model last loss:  1.558424711227417

Epoch: 0, accuracy:0.1215,  loss:2.5331,  val_accuracy:0.4516,  val_loss:1.9590,  
...............................................adapt model last loss:  0.9564798474311829

Epoch: 0, accuracy:0.2324,  loss:2.1410,  val_accuracy:0.2927,  val_loss:1.7654,  
....

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')




```
reporting k Folds average
np.mean(arr_dl) =  0.5787729004171958
np.mean(arr_knn) =  0.5019907491384001
np.mean(arr_retrieval_loss) =  0.519485760928714
np.mean(arr_retrieval_N_adapt_loss) =  0.4629920188645021
np.mean(arr_normal_cdh_loss) =  0.4009341556321422
np.mean(retrieval_N_EAC_adapt_loss) =  0.4960003627788863
np.mean(C2C_EAC_NN_CDH_loss) =  0.6081081081081081

reporting k Folds balanced
np.mean(arr_dl) =  0.497038232059322
np.mean(arr_knn) =  0.4899373868630107
np.mean(arr_retrieval_loss) =  0.48990861672406955
np.mean(arr_retrieval_N_adapt_loss) =  0.41667873642559367
np.mean(arr_normal_cdh_loss) =  0.36369741344219764
np.mean(retrieval_N_EAC_adapt_loss) =  0.4607537585037126
np.mean(C2C_EAC_NN_CDH_loss) =  0.4867682562262906
```



multi runs

```
reporting k Folds average
np.mean(arr_dl) =  0.596252494104843
np.mean(arr_knn) =  0.5028369308906221
np.mean(arr_retrieval_loss) =  0.5350562307273716
np.mean(arr_retrieval_N_adapt_loss) =  0.47281516415744596
np.mean(arr_normal_cdh_loss) =  0.41957826954471245
np.mean(retrieval_N_EAC_adapt_loss) =  0.49868764737892257
np.mean(C2C_EAC_NN_CDH_loss) =  0.5833085434427716

reporting std average
np.std(arr_dl) =  0.041162097045133426
np.std(arr_knn) =  0.044034608484149425
np.std(arr_retrieval_loss) =  0.042307874841011034
np.std(arr_retrieval_N_adapt_loss) =  0.03892795817498548
np.std(arr_normal_cdh_loss) =  0.04360502350468035
np.std(retrieval_N_EAC_adapt_loss) =  0.03727221642946003
np.std(C2C_EAC_NN_CDH_loss) =  0.0402225769494696

reporting k Folds balanced
np.mean(arr_dl) =  0.5413748609381609
np.mean(arr_knn) =  0.5076535748654931
np.mean(arr_retrieval_loss) =  0.5126662977566607
np.mean(arr_retrieval_N_adapt_loss) =  0.44937894844298165
np.mean(arr_normal_cdh_loss) =  0.4060282335696581
np.mean(retrieval_N_EAC_adapt_loss) =  0.4831764787878637
np.mean(C2C_EAC_NN_CDH_loss) =  0.5510401752285099

reporting std balanced
np.std(arr_dl) =  0.07455598914503703
np.std(arr_knn) =  0.06258286037550173
np.std(arr_retrieval_loss) =  0.06872053852069726
np.std(arr_retrieval_N_adapt_loss) =  0.06714262135835385
np.std(arr_normal_cdh_loss) =  0.08040804958316043
np.std(retrieval_N_EAC_adapt_loss) =  0.06759348735481192
np.std(C2C_EAC_NN_CDH_loss) =  0.06323182419886726

```



# Dataset: Seeds

In [22]:
filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/DL-CDH for classification/seeds_dataset.txt'
nominalCols = [] 
numericCols = [0,1,2,3,4,5,6]
def GetDataMatrix():
    
    # Data frame with make and model
    X = pd.read_csv(filename,header=None, usecols=(0,1,2,3,4,5,6,),delim_whitespace=True);

    y = pd.read_csv(filename,header=None, usecols=(7,),delim_whitespace=True);

    #no missing values
    #remove rows with ? values
    # if(X.values == '?'):
    #   rows_with_na = (X.values == '?').any(1)
    #   X = X[~rows_with_na]
    #   y = y[~rows_with_na]

    X, y = shuffle(X, y)
    print(X.head(0))
    # Turns categorical data into binary values across many columns
    #special one hot encoding with multiple values
    # market_category_dummies = X['Market Category'].str.get_dummies(sep=',')
    # X = pd.concat([X, market_category_dummies], axis=1)
    # X = X.drop(columns=['Market Category'])
    #normal one hot encoding
    X = pd.get_dummies(X, dummy_na = False, columns=nominalCols );
    y = pd.get_dummies(y, dummy_na=False, columns=[7])
    # Fill the null values with zeros
    # X.fillna(0, inplace=True);
    #there shouldn't be null, since it's already cleaned.
    print(X.isnull().sum())
    X = scaleX(X,numericCols)

    return (X, y)

##########

(X, y) = GetDataMatrix() #Gets the X,Y
num_classes = len(y.columns)

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6]
Index: []
0    0
1    0
2    0
3    0
4    0
5    0
6    0
dtype: int64


In [23]:
X.head(3)

,0,1,2,3,4,5,6
159,-1.156665,-1.027970,-1.896169,-0.734226,-1.495711,1.125220,-0.200020
179,-0.908625,-0.836082,-1.090166,-0.614319,-0.970252,-1.359492,-0.469238
188,-1.246235,-1.334990,-0.492026,-1.220643,-1.161328,2.549245,-0.919974


In [24]:
X.shape

(210, 7)

# Trying 10 fold: Seeds

In [25]:
from sklearn.model_selection import KFold

#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
# random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

kFoldExperiment(X,y,5)


Epoch: 0, accuracy:0.6294,  loss:0.8213,  val_accuracy:0.8947,  val_loss:0.4791,  
....................................................................................................
Epoch: 100, accuracy:1.0000,  loss:0.0120,  val_accuracy:1.0000,  val_loss:0.0002,  
....................................................................................................
Epoch: 200, accuracy:1.0000,  loss:0.0026,  val_accuracy:1.0000,  val_loss:0.0000,  
....................................................................................................
Epoch: 300, accuracy:1.0000,  loss:0.0009,  val_accuracy:1.0000,  val_loss:0.0000,  
....................................................................................................
Epoch: 400, accuracy:1.0000,  loss:0.0004,  val_accuracy:1.0000,  val_loss:0.0000,  
...................................................dl last loss:  4.0154654357138497e-07

Epoch: 0, accuracy:0.7066,  loss:0.6509,  val_accuracy:0.8049,  val_loss:0.4399,  



```
reporting k Folds average
np.mean(arr_dl) =  0.8952380952380953
np.mean(arr_knn) =  0.880952380952381
np.mean(arr_retrieval_loss) =  0.8904761904761905
np.mean(arr_retrieval_N_adapt_loss) =  0.9
np.mean(arr_normal_cdh_loss) =  0.8857142857142858
np.mean(retrieval_N_EAC_adapt_loss) =  0.9
np.mean(C2C_EAC_NN_CDH_loss) =  0.9523809523809523

reporting k Folds balanced
np.mean(arr_dl) =  0.8982647907647909
np.mean(arr_knn) =  0.8878270803270801
np.mean(arr_retrieval_loss) =  0.89241822991823
np.mean(arr_retrieval_N_adapt_loss) =  0.9047462722462722
np.mean(arr_normal_cdh_loss) =  0.8898785473785473
np.mean(retrieval_N_EAC_adapt_loss) =  0.9039526214526215
np.mean(C2C_EAC_NN_CDH_loss) =  0.9444444444444445
```



Multi runs



```
reporting k Folds average
np.mean(arr_dl) =  0.9495238095238094
np.mean(arr_knn) =  0.9209523809523809
np.mean(arr_retrieval_loss) =  0.9333333333333331
np.mean(arr_retrieval_N_adapt_loss) =  0.9428571428571427
np.mean(arr_normal_cdh_loss) =  0.8638095238095239
np.mean(retrieval_N_EAC_adapt_loss) =  0.9533333333333333
np.mean(C2C_EAC_NN_CDH_loss) =  0.9485714285714285

reporting std average
np.std(arr_dl) =  0.04986148614365114
np.std(arr_knn) =  0.053545496369030846
np.std(arr_retrieval_loss) =  0.04761904761904762
np.std(arr_retrieval_N_adapt_loss) =  0.05387480237611791
np.std(arr_normal_cdh_loss) =  0.0719094921181435
np.std(retrieval_N_EAC_adapt_loss) =  0.049934197062546024
np.std(C2C_EAC_NN_CDH_loss) =  0.05114560602703829

reporting k Folds balanced
np.mean(arr_dl) =  0.9531132756132756
np.mean(arr_knn) =  0.9259126244126245
np.mean(arr_retrieval_loss) =  0.9365227735227737
np.mean(arr_retrieval_N_adapt_loss) =  0.9475553150553152
np.mean(arr_normal_cdh_loss) =  0.8667980722980723
np.mean(retrieval_N_EAC_adapt_loss) =  0.9563701298701298
np.mean(C2C_EAC_NN_CDH_loss) =  0.9527435527435528

reporting std balanced
np.std(arr_dl) =  0.045773947048098146
np.std(arr_knn) =  0.05110608134934581
np.std(arr_retrieval_loss) =  0.048890137004640136
np.std(arr_retrieval_N_adapt_loss) =  0.0483200172401884
np.std(arr_normal_cdh_loss) =  0.0728159002043527
np.std(retrieval_N_EAC_adapt_loss) =  0.04645073407012013
np.std(C2C_EAC_NN_CDH_loss) =  0.04559015734843396
```



# Dataset: Pima


In [26]:
filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/DL-CDH for classification/pima-indians-diabetes.csv'
nominalCols = [] 
numericCols = [0,1,2,3,4,5,6,7]
def GetDataMatrix():
    
    # Data frame with make and model
    X = pd.read_csv(filename,header=None, usecols=(0,1,2,3,4,5,6,7,));

    y = pd.read_csv(filename,header=None, usecols=(8,));

    #no missing values
    #remove rows with ? values
    # if(X.values == '?'):
    #   rows_with_na = (X.values == '?').any(1)
    #   X = X[~rows_with_na]
    #   y = y[~rows_with_na]

    X, y = shuffle(X, y)
    print(X.head(0))
    # Turns categorical data into binary values across many columns
    #special one hot encoding with multiple values
    # market_category_dummies = X['Market Category'].str.get_dummies(sep=',')
    # X = pd.concat([X, market_category_dummies], axis=1)
    # X = X.drop(columns=['Market Category'])
    #normal one hot encoding
    X = pd.get_dummies(X, dummy_na = False, columns=nominalCols );
    y = pd.get_dummies(y, dummy_na=False, columns=[8])
    # Fill the null values with zeros
    # X.fillna(0, inplace=True);
    #there shouldn't be null, since it's already cleaned.
    print(X.isnull().sum())
    X = scaleX(X,numericCols)

    return (X, y)

##########

(X, y) = GetDataMatrix() #Gets the X,Y
num_classes = len(y.columns)

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7]
Index: []
0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
dtype: int64


In [ ]:
X.head(4)

,0,1,2,3,4,5,6,7
532,-0.844885,-1.092099,-0.160546,1.973647,-0.128502,1.181291,1.344326,-0.360847
703,-0.547919,0.253678,-3.572597,-1.288212,-0.692891,0.825917,-0.507006,0.660206
432,-0.844885,-1.279882,0.253036,-0.598204,-0.171917,-0.252897,0.166480,-0.956462
405,-0.547919,0.065895,-1.091105,0.719086,0.739787,1.282826,0.145339,-0.616111


In [ ]:
y.head(4)

,8_0,8_1
532,1,0
703,1,0
432,1,0
405,1,0


# Trying 10 fold: Pima

In [27]:
from sklearn.model_selection import KFold

#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
# random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

kFoldExperiment(X,y,5)


Epoch: 0, accuracy:0.6989,  loss:0.6034,  val_accuracy:0.7286,  val_loss:0.4569,  
.............dl last loss:  0.48418474197387695

Epoch: 0, accuracy:0.7356,  loss:0.5232,  val_accuracy:0.7500,  val_loss:0.4790,  
................................adapt model last loss:  0.3547195494174957

Epoch: 0, accuracy:0.7000,  loss:0.5774,  val_accuracy:0.7891,  val_loss:0.4486,  
................adapt model last loss:  0.4056171178817749

dl_loss 0.6883116883116883
knn_loss 0.7142857142857143
retrieval_loss 0.7012987012987013
retrieval_N_adapt_loss 0.6103896103896104
CBR normal CDH loss 0.6493506493506493
retrieval_N_EAC_adapt_loss loss 0.6493506493506493
C2C_EAC_NN_CDH_loss  0.6753246753246753

Epoch: 0, accuracy:0.6554,  loss:0.6234,  val_accuracy:0.7143,  val_loss:0.5006,  
...........dl last loss:  0.5214617848396301

Epoch: 0, accuracy:0.7181,  loss:0.5505,  val_accuracy:0.7491,  val_loss:0.4833,  
......................................adapt model last loss:  0.3003102242946625

Epoch: 0,



```
reporting k Folds average
np.mean(arr_dl) =  0.5536397812713602
np.mean(arr_knn) =  0.6849794941900205
np.mean(arr_retrieval_loss) =  0.6706425153793576
np.mean(arr_retrieval_N_adapt_loss) =  0.5987696514012303
np.mean(arr_normal_cdh_loss) =  0.6042207792207792
np.mean(retrieval_N_EAC_adapt_loss) =  0.583116883116883
np.mean(C2C_EAC_NN_CDH_loss) =  0.6842105263157895

reporting k Folds balanced
np.mean(arr_dl) =  0.5916919162258714
np.mean(arr_knn) =  0.6463337392668883
np.mean(arr_retrieval_loss) =  0.6444572815245712
np.mean(arr_retrieval_N_adapt_loss) =  0.5808719461296186
np.mean(arr_normal_cdh_loss) =  0.5773530645523316
np.mean(retrieval_N_EAC_adapt_loss) =  0.5613729885269192
np.mean(C2C_EAC_NN_CDH_loss) =  0.7043478260869566
```



multi runs


```
reporting k Folds average
np.mean(arr_dl) =  0.7616814764183185
np.mean(arr_knn) =  0.7388072453861928
np.mean(arr_retrieval_loss) =  0.7039131920710866
np.mean(arr_retrieval_N_adapt_loss) =  0.687758031442242
np.mean(arr_normal_cdh_loss) =  0.6458612440191388
np.mean(retrieval_N_EAC_adapt_loss) =  0.694788106630212
np.mean(C2C_EAC_NN_CDH_loss) =  0.7216370471633629

reporting std average
np.std(arr_dl) =  0.0453517428900051
np.std(arr_knn) =  0.04288425528615694
np.std(arr_retrieval_loss) =  0.0483295310141565
np.std(arr_retrieval_N_adapt_loss) =  0.04834789886788082
np.std(arr_normal_cdh_loss) =  0.045993851832649944
np.std(retrieval_N_EAC_adapt_loss) =  0.05250339747400515
np.std(C2C_EAC_NN_CDH_loss) =  0.050103082605795816

reporting k Folds balanced
np.mean(arr_dl) =  0.7300267664938329
np.mean(arr_knn) =  0.7030537273011396
np.mean(arr_retrieval_loss) =  0.6656147236655896
np.mean(arr_retrieval_N_adapt_loss) =  0.6565536802548944
np.mean(arr_normal_cdh_loss) =  0.6130391237107407
np.mean(retrieval_N_EAC_adapt_loss) =  0.6660781715675492
np.mean(C2C_EAC_NN_CDH_loss) =  0.6974413636611423

reporting std balanced
np.std(arr_dl) =  0.04704098129329441
np.std(arr_knn) =  0.04668937207540498
np.std(arr_retrieval_loss) =  0.05305052625028058
np.std(arr_retrieval_N_adapt_loss) =  0.055723330990326
np.std(arr_normal_cdh_loss) =  0.05135387729815293
np.std(retrieval_N_EAC_adapt_loss) =  0.05806275201335232
np.std(C2C_EAC_NN_CDH_loss) =  0.05060338898805063
```



# Dataset: Page-blocks

In [18]:
filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/DL-CDH for classification/page-blocks.data'
nominalCols = [] 
numericCols = [0,1,2,3,4,5,6,7,8,9]
def GetDataMatrix():
    
    # Data frame with make and model
    X = pd.read_csv(filename,header=None, usecols=(0,1,2,3,4,5,6,7,8,9,),delim_whitespace=True);

    y = pd.read_csv(filename,header=None, usecols=(10,),delim_whitespace=True);

    #no missing values
    #remove rows with ? values
    # if(X.values == '?'):
    #   rows_with_na = (X.values == '?').any(1)
    #   X = X[~rows_with_na]
    #   y = y[~rows_with_na]

    X, y = shuffle(X, y)
    print(X.head(0))
    # Turns categorical data into binary values across many columns
    #special one hot encoding with multiple values
    # market_category_dummies = X['Market Category'].str.get_dummies(sep=',')
    # X = pd.concat([X, market_category_dummies], axis=1)
    # X = X.drop(columns=['Market Category'])
    #normal one hot encoding
    X = pd.get_dummies(X, dummy_na = False, columns=nominalCols );
    y = pd.get_dummies(y, dummy_na=False, columns=[10])
    # Fill the null values with zeros
    # X.fillna(0, inplace=True);
    #there shouldn't be null, since it's already cleaned.
    print(X.isnull().sum())
    X = scaleX(X,numericCols)

    return (X, y)

##########

(X, y) = GetDataMatrix() #Gets the X,Y
num_classes = len(y.columns)

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Index: []
0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
dtype: int64


In [19]:
X.shape

(5473, 10)

# Trying 10 fold: Page-blocks

In [20]:
from sklearn.model_selection import KFold

#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
# random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

kFoldExperiment(X,y,2)


Epoch: 0, accuracy:0.9057,  loss:0.4920,  val_accuracy:0.9736,  val_loss:0.1497,  
..............................dl last loss:  0.04325513541698456

Epoch: 0, accuracy:0.9058,  loss:0.2872,  val_accuracy:0.9386,  val_loss:0.1699,  
.............................................adapt model last loss:  0.03737979754805565

Epoch: 0, accuracy:0.7266,  loss:0.8532,  val_accuracy:0.8165,  val_loss:0.4800,  
..................................adapt model last loss:  0.17723314464092255

Epoch: 0, accuracy:0.4918,  loss:1.9954,  val_accuracy:0.7500,  val_loss:0.6741,  
....................................adapt model last loss:  0.1108398288488388

Epoch: 0, accuracy:0.6271,  loss:1.0379,  val_accuracy:0.7500,  val_loss:0.7237,  
.......................................adapt model last loss:  0.4583685100078583

Epoch: 0, accuracy:0.6897,  loss:0.9969,  val_accuracy:0.7922,  val_loss:0.7591,  
.........................................adapt model last loss:  0.3808254897594452

dl_loss 0.95985401



```
reporting k Folds average
np.mean(arr_dl) =  0.9539538824910927
np.mean(arr_knn) =  0.956696112838442
np.mean(arr_retrieval_loss) =  0.9579738187058808
np.mean(arr_retrieval_N_adapt_loss) =  0.9349567648353995
np.mean(arr_normal_cdh_loss) =  0.9422583701410481
np.mean(retrieval_N_EAC_adapt_loss) =  0.9395231454916665
np.mean(C2C_EAC_NN_CDH_loss) =  0.9414990859232175

reporting k Folds balanced
np.mean(arr_dl) =  0.712117232970063
np.mean(arr_knn) =  0.7138370403732355
np.mean(arr_retrieval_loss) =  0.7633803995289032
np.mean(arr_retrieval_N_adapt_loss) =  0.6307572269922996
np.mean(arr_normal_cdh_loss) =  0.6442186052273662
np.mean(retrieval_N_EAC_adapt_loss) =  0.7034092808757572
np.mean(C2C_EAC_NN_CDH_loss) =  0.8771153846153845
```



2 times 10 fold

```
reporting k Folds average
np.mean(arr_dl) =  0.9685731061263159
np.mean(arr_knn) =  0.9665633048212545
np.mean(arr_retrieval_loss) =  0.9646435767757776
np.mean(arr_retrieval_N_adapt_loss) =  0.9489289622226076
np.mean(arr_normal_cdh_loss) =  0.9473763661111038
np.mean(retrieval_N_EAC_adapt_loss) =  0.9515787840777165
np.mean(C2C_EAC_NN_CDH_loss) =  0.9675647860259678

reporting std average
np.std(arr_dl) =  0.007996656107944169
np.std(arr_knn) =  0.007818635769009496
np.std(arr_retrieval_loss) =  0.008161736488200913
np.std(arr_retrieval_N_adapt_loss) =  0.011805093942928934
np.std(arr_normal_cdh_loss) =  0.009689588811481999
np.std(retrieval_N_EAC_adapt_loss) =  0.010289273047720357
np.std(C2C_EAC_NN_CDH_loss) =  0.007179361150150107

reporting k Folds balanced
np.mean(arr_dl) =  0.8100202967765615
np.mean(arr_knn) =  0.8269490132087848
np.mean(arr_retrieval_loss) =  0.8404361865499871
np.mean(arr_retrieval_N_adapt_loss) =  0.7361241938606605
np.mean(arr_normal_cdh_loss) =  0.7320485782912092
np.mean(retrieval_N_EAC_adapt_loss) =  0.8012843396089165
np.mean(C2C_EAC_NN_CDH_loss) =  0.8565965349436047

reporting std balanced
np.std(arr_dl) =  0.07519104176013881
np.std(arr_knn) =  0.061361255634633245
np.std(arr_retrieval_loss) =  0.05321437240986915
np.std(arr_retrieval_N_adapt_loss) =  0.0985919717981357
np.std(arr_normal_cdh_loss) =  0.10249295752673333
np.std(retrieval_N_EAC_adapt_loss) =  0.072499686661438
np.std(C2C_EAC_NN_CDH_loss) =  0.06065210656000591

```



# Dataset: Contraceptive Method Choice Data Set

In [21]:
filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/DL-CDH for classification/cmc.data'
nominalCols = [1,2,6,7] 
numericCols = [0,3,4,5,8]
def GetDataMatrix():
    
    # Data frame with make and model
    X = pd.read_csv(filename,header=None, usecols=(0,1,2,3,4,5,6,7,8,),sep=',');

    y = pd.read_csv(filename,header=None, usecols=(9,),sep=',');

    #no missing values
    #remove rows with ? values
    # if(X.values == '?'):
    #   rows_with_na = (X.values == '?').any(1)
    #   X = X[~rows_with_na]
    #   y = y[~rows_with_na]

    X, y = shuffle(X, y)
    print(X.head(0))
    # Turns categorical data into binary values across many columns
    #special one hot encoding with multiple values
    # market_category_dummies = X['Market Category'].str.get_dummies(sep=',')
    # X = pd.concat([X, market_category_dummies], axis=1)
    # X = X.drop(columns=['Market Category'])
    #normal one hot encoding
    X = pd.get_dummies(X, dummy_na = False, columns=nominalCols );
    y = pd.get_dummies(y, dummy_na=False, columns=[9])
    # Fill the null values with zeros
    # X.fillna(0, inplace=True);
    #there shouldn't be null, since it's already cleaned.
    print(X.isnull().sum())
    X = scaleX(X,numericCols)

    return (X, y)

##########

(X, y) = GetDataMatrix() #Gets the X,Y
num_classes = len(y.columns)

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8]
Index: []
0      0
3      0
4      0
5      0
8      0
1_1    0
1_2    0
1_3    0
1_4    0
2_1    0
2_2    0
2_3    0
2_4    0
6_1    0
6_2    0
6_3    0
6_4    0
7_1    0
7_2    0
7_3    0
7_4    0
dtype: int64


In [ ]:
X.head(4)

,0,3,4,5,8,1_1,1_2,1_3,1_4,2_1,2_2,2_3,2_4,6_1,6_2,6_3,6_4,7_1,7_2,7_3,7_4
932,-0.430224,0.313277,0.419021,0.578134,-0.282687,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0
1240,-0.065458,0.313277,0.419021,-1.729702,-0.282687,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0
60,1.150428,2.858078,0.419021,0.578134,-0.282687,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0
1096,-1.038167,-0.534990,0.419021,0.578134,-0.282687,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1


In [22]:
X.shape

(1473, 21)

In [ ]:
y.head(4)

,9_1,9_2,9_3
1461,0,0,1
462,0,1,0
271,1,0,0
200,1,0,0


# Trying 10 fold: Contraceptive

In [23]:
from sklearn.model_selection import KFold

#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
# random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

kFoldExperiment(X,y,5)


Epoch: 0, accuracy:0.4211,  loss:1.0901,  val_accuracy:0.4737,  val_loss:1.0050,  
.................dl last loss:  0.9596326947212219

Epoch: 0, accuracy:0.4895,  loss:1.0037,  val_accuracy:0.5297,  val_loss:0.9337,  
...................................adapt model last loss:  0.7754613757133484

Epoch: 0, accuracy:0.4798,  loss:1.0003,  val_accuracy:0.4660,  val_loss:1.0187,  
..............adapt model last loss:  1.081352949142456

Epoch: 0, accuracy:0.4885,  loss:1.0077,  val_accuracy:0.4901,  val_loss:0.9902,  
.........................adapt model last loss:  0.9113894104957581

dl_loss 0.5
knn_loss 0.33783783783783783
retrieval_loss 0.42567567567567566
retrieval_N_adapt_loss 0.44594594594594594
CBR normal CDH loss 0.3783783783783784
retrieval_N_EAC_adapt_loss loss 0.47297297297297297
C2C_EAC_NN_CDH_loss  0.5135135135135135

Epoch: 0, accuracy:0.4396,  loss:1.0548,  val_accuracy:0.4436,  val_loss:0.9788,  
............dl last loss:  0.9805692434310913

Epoch: 0, accuracy:0.4813,  l



```
reporting k Folds average
np.mean(arr_dl) =  0.5288656002941717
np.mean(arr_knn) =  0.4256894649751793
np.mean(arr_retrieval_loss) =  0.4500873322301894
np.mean(arr_retrieval_N_adapt_loss) =  0.49218606361463496
np.mean(arr_normal_cdh_loss) =  0.39440614083471226
np.mean(retrieval_N_EAC_adapt_loss) =  0.49153337010479864
np.mean(C2C_EAC_NN_CDH_loss) =  0.54421768707483
reporting k Folds balanced
np.mean(arr_dl) =  0.49578508188150217
np.mean(arr_knn) =  0.44855540994875354
np.mean(arr_retrieval_loss) =  0.43454317477251464
np.mean(arr_retrieval_N_adapt_loss) =  0.4699601045679717
np.mean(arr_normal_cdh_loss) =  0.3871058606330141
np.mean(retrieval_N_EAC_adapt_loss) =  0.47140368648915293
np.mean(C2C_EAC_NN_CDH_loss) =  0.5100198412698412
```



Multi run

```
reporting k Folds average
np.mean(arr_dl) =  0.547317521603236
np.mean(arr_knn) =  0.4096240117668689
np.mean(arr_retrieval_loss) =  0.4347416804559661
np.mean(arr_retrieval_N_adapt_loss) =  0.4794392351535209
np.mean(arr_normal_cdh_loss) =  0.3880400808972238
np.mean(retrieval_N_EAC_adapt_loss) =  0.47820922963780105
np.mean(C2C_EAC_NN_CDH_loss) =  0.513914322485751

reporting std average
np.std(arr_dl) =  0.03810123623451787
np.std(arr_knn) =  0.03750680710336985
np.std(arr_retrieval_loss) =  0.03741967681925101
np.std(arr_retrieval_N_adapt_loss) =  0.04098959878103348
np.std(arr_normal_cdh_loss) =  0.03614251947842028
np.std(retrieval_N_EAC_adapt_loss) =  0.04481613497029232
np.std(C2C_EAC_NN_CDH_loss) =  0.043132229423507876

reporting k Folds balanced
np.mean(arr_dl) =  0.5229381141508201
np.mean(arr_knn) =  0.43883572984731123
np.mean(arr_retrieval_loss) =  0.4178267704715454
np.mean(arr_retrieval_N_adapt_loss) =  0.4548793652709406
np.mean(arr_normal_cdh_loss) =  0.3839333954072596
np.mean(retrieval_N_EAC_adapt_loss) =  0.45469448089499553
np.mean(C2C_EAC_NN_CDH_loss) =  0.4969222855097253

reporting std balanced
np.std(arr_dl) =  0.038280534421347635
np.std(arr_knn) =  0.033924939097700425
np.std(arr_retrieval_loss) =  0.03649325651929315
np.std(arr_retrieval_N_adapt_loss) =  0.041709425475082475
np.std(arr_normal_cdh_loss) =  0.03847498751013724
np.std(retrieval_N_EAC_adapt_loss) =  0.04518575093262454
np.std(C2C_EAC_NN_CDH_loss) =  0.04314633544083794
```



# Dataset: Abalone

In [ ]:
filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/DL-CDH for classification/abalone.data'
nominalCols = [0] 
numericCols = [1,2,3,4,5,6,7]
def GetDataMatrix():
    
    # Data frame with make and model
    X = pd.read_csv(filename,header=None, usecols=(0,1,2,3,4,5,6,7,),sep=',');

    y = pd.read_csv(filename,header=None, usecols=(8,),sep=',');

    #no missing values
    #remove rows with ? values
    # if(X.values == '?'):
    #   rows_with_na = (X.values == '?').any(1)
    #   X = X[~rows_with_na]
    #   y = y[~rows_with_na]

    X, y = shuffle(X, y)
    print(X.head(0))
    # Turns categorical data into binary values across many columns
    #special one hot encoding with multiple values
    # market_category_dummies = X['Market Category'].str.get_dummies(sep=',')
    # X = pd.concat([X, market_category_dummies], axis=1)
    # X = X.drop(columns=['Market Category'])
    #normal one hot encoding
    X = pd.get_dummies(X, dummy_na = False, columns=nominalCols );
    y = pd.get_dummies(y, dummy_na=False, columns=[8])
    # Fill the null values with zeros
    # X.fillna(0, inplace=True);
    #there shouldn't be null, since it's already cleaned.
    print(X.isnull().sum())
    X = scaleX(X,numericCols)

    return (X, y)

##########

(X, y) = GetDataMatrix() #Gets the X,Y
num_classes = len(y.columns)

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7]
Index: []
1      0
2      0
3      0
4      0
5      0
6      0
7      0
0_F    0
0_I    0
0_M    0
dtype: int64


# Trying 10 fold: Abalone

In [ ]:
from sklearn.model_selection import KFold

#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
# random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

kFoldExperiment(X,y)


Epoch: 0, accuracy:0.1960,  loss:2.6164,  val_accuracy:0.2154,  val_loss:2.3411,  
..........................................dl last loss:  1.9874540567398071

Epoch: 0, accuracy:0.0303,  loss:3.4793,  val_accuracy:0.0000,  val_loss:3.1847,  
....................adapt model last loss:  3.142123222351074

Epoch: 0, accuracy:0.0556,  loss:3.5281,  val_accuracy:0.0000,  val_loss:3.1774,  
...........adapt model last loss:  3.364961624145508

Epoch: 0, accuracy:0.0234,  loss:3.3722,  val_accuracy:0.0357,  val_loss:3.3093,  
.............................................................................................adapt model last loss:  1.828969955444336

Epoch: 0, accuracy:0.0892,  loss:3.1747,  val_accuracy:0.0755,  val_loss:3.0242,  
..........................................................................................adapt model last loss:  1.5212504863739014

Epoch: 0, accuracy:0.1028,  loss:3.0690,  val_accuracy:0.1383,  val_loss:2.7743,  
.....................................

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')



dl_loss 0.28708133971291866
knn_loss 0.10526315789473684
retrieval_loss 0.17464114832535885
retrieval_N_adapt_loss 0.16507177033492823
CBR normal CDH loss 0.17464114832535885
retrieval_N_EAC_adapt_loss loss 0.1507177033492823
C2C_EAC_NN_CDH_loss  0.17464114832535885

Epoch: 0, accuracy:0.1850,  loss:2.5610,  val_accuracy:0.2500,  val_loss:2.2074,  
.................................................................dl last loss:  1.8925399780273438


ValueError: ignored

# Dataset: Wine-white

In [26]:
filename = '/content/drive/My Drive/2020 Summer Assistantship CBR+Deep Learning/DL-CDH for classification/winequality-white.csv'
nominalCols = [] 
numericCols = [0,1,2,3,4,5,6,7,8,9,10]
def GetDataMatrix():
    
    # Data frame with make and model
    X = pd.read_csv(filename,header=0, names=[0,1,2,3,4,5,6,7,8,9,10,11], usecols=(0,1,2,3,4,5,6,7,8,9,10,),sep=';');

    y = pd.read_csv(filename,header=0, names=[0,1,2,3,4,5,6,7,8,9,10,11], usecols=(11,),sep=';');

    #no missing values
    #remove rows with ? values
    # if(X.values == '?'):
    #   rows_with_na = (X.values == '?').any(1)
    #   X = X[~rows_with_na]
    #   y = y[~rows_with_na]

    X, y = shuffle(X, y)
    print(X.head(0))
    # Turns categorical data into binary values across many columns
    #special one hot encoding with multiple values
    # market_category_dummies = X['Market Category'].str.get_dummies(sep=',')
    # X = pd.concat([X, market_category_dummies], axis=1)
    # X = X.drop(columns=['Market Category'])
    #normal one hot encoding
    X = pd.get_dummies(X, dummy_na = False, columns=nominalCols );
    y = pd.get_dummies(y, dummy_na=False, columns=[11])
    # Fill the null values with zeros
    # X.fillna(0, inplace=True);
    #there shouldn't be null, since it's already cleaned.
    print(X.isnull().sum())
    X = scaleX(X,numericCols)

    return (X, y)

##########

(X, y) = GetDataMatrix() #Gets the X,Y
num_classes = len(y.columns)

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Index: []
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
dtype: int64


In [27]:
X.shape

(4898, 11)

# Trying 10 fold: Wine-white

In [28]:
from sklearn.model_selection import KFold

#ALL PARAMETERS HERE
#0, no EAC
#1, adapt from multiple neighbors
#2, adapt from multiple neighbors, each from a different class.
EAC_adapt = "12"
#1, rules from nearest pairs
#2, rules from random pairs
#12, rules from both nearest pairs and random pairs
#124, rules from both nearest pairs and random pairs, with designated number of random pairs
pair_selection = "145"
# random_pairs_count = 6000
#1, pair from partial knowledge
#2, pair from full knowledge
pair_knowledge = 1

kFoldExperiment(X,y,2)


Epoch: 0, accuracy:0.4832,  loss:1.2732,  val_accuracy:0.5624,  val_loss:1.1777,  
...............dl last loss:  1.1584621667861938

Epoch: 0, accuracy:0.2276,  loss:2.0631,  val_accuracy:0.2353,  val_loss:1.8987,  
.............adapt model last loss:  1.5769753456115723

Epoch: 0, accuracy:0.3183,  loss:1.7953,  val_accuracy:0.3516,  val_loss:1.6383,  
.....................adapt model last loss:  1.2972246408462524

Epoch: 0, accuracy:0.4362,  loss:1.3647,  val_accuracy:0.5394,  val_loss:1.1354,  
.....................................................................................adapt model last loss:  0.39547887444496155

Epoch: 0, accuracy:0.4412,  loss:1.3436,  val_accuracy:0.5236,  val_loss:1.1405,  
.........................................................................................adapt model last loss:  0.11935800313949585

Epoch: 0, accuracy:0.4259,  loss:1.4227,  val_accuracy:0.4912,  val_loss:1.2163,  
..........................................................adapt m

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')



dl_loss 0.6142857142857143
knn_loss 0.5428571428571428
retrieval_loss 0.636734693877551
retrieval_N_adapt_loss 0.5551020408163265
CBR normal CDH loss 0.4857142857142857
retrieval_N_EAC_adapt_loss loss 0.610204081632653
C2C_EAC_NN_CDH_loss  0.6591836734693878

Epoch: 0, accuracy:0.4847,  loss:1.2879,  val_accuracy:0.5510,  val_loss:1.1733,  
...................dl last loss:  1.1051769256591797

Epoch: 0, accuracy:0.2338,  loss:2.5366,  val_accuracy:0.1111,  val_loss:2.2159,  
.............adapt model last loss:  2.172363519668579

Epoch: 0, accuracy:0.3501,  loss:1.6931,  val_accuracy:0.4046,  val_loss:1.3834,  
..................adapt model last loss:  1.257591962814331

Epoch: 0, accuracy:0.4342,  loss:1.3868,  val_accuracy:0.4830,  val_loss:1.2137,  
.........................................................................................adapt model last loss:  0.377341091632843

Epoch: 0, accuracy:0.4586,  loss:1.3113,  val_accuracy:0.5021,  val_loss:1.1367,  
.....................

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


...

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')



dl_loss 0.5664621676891616
knn_loss 0.558282208588957
retrieval_loss 0.6789366053169734
retrieval_N_adapt_loss 0.5214723926380368
CBR normal CDH loss 0.4785276073619632
retrieval_N_EAC_adapt_loss loss 0.6094069529652352
C2C_EAC_NN_CDH_loss  0.6319018404907976

Epoch: 0, accuracy:0.4992,  loss:1.2792,  val_accuracy:0.5351,  val_loss:1.1716,  
............................dl last loss:  1.0985716581344604

Epoch: 0, accuracy:0.1875,  loss:2.4886,  val_accuracy:0.2000,  val_loss:2.0448,  
............adapt model last loss:  2.488844394683838

Epoch: 0, accuracy:0.3180,  loss:1.7645,  val_accuracy:0.3333,  val_loss:1.5891,  
.........................................adapt model last loss:  1.279021143913269

Epoch: 0, accuracy:0.4385,  loss:1.3796,  val_accuracy:0.5090,  val_loss:1.1786,  
....................................................................adapt model last loss:  0.4764101207256317

Epoch: 0, accuracy:0.4588,  loss:1.3116,  val_accuracy:0.5160,  val_loss:1.1472,  
.........

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')



dl_loss 0.6073619631901841
knn_loss 0.5623721881390593
retrieval_loss 0.6830265848670757
retrieval_N_adapt_loss 0.5807770961145194
CBR normal CDH loss 0.48466257668711654
retrieval_N_EAC_adapt_loss loss 0.6114519427402862
C2C_EAC_NN_CDH_loss  0.6809815950920245

reporting k Folds average
np.mean(arr_dl) =  0.5763580401485748
np.mean(arr_knn) =  0.5425783564959727
np.mean(arr_retrieval_loss) =  0.6569103960602646
np.mean(arr_retrieval_N_adapt_loss) =  0.5504342473185594
np.mean(arr_normal_cdh_loss) =  0.46938107758440795
np.mean(retrieval_N_EAC_adapt_loss) =  0.6035167563958099
np.mean(C2C_EAC_NN_CDH_loss) =  0.6354741037519303

reporting std average
np.std(arr_dl) =  0.02179267686442935
np.std(arr_knn) =  0.02165176555679546
np.std(arr_retrieval_loss) =  0.022611759724934604
np.std(arr_retrieval_N_adapt_loss) =  0.024432698325331216
np.std(arr_normal_cdh_loss) =  0.01985915156665192
np.std(retrieval_N_EAC_adapt_loss) =  0.018426137178453882
np.std(C2C_EAC_NN_CDH_loss) =  0.02697159808

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1859: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')




```
reporting k Folds average
np.mean(arr_dl) =  0.4665381244522349
np.mean(arr_knn) =  0.4697909102291223
np.mean(arr_retrieval_loss) =  0.5979950753307457
np.mean(arr_retrieval_N_adapt_loss) =  0.31564959726221775
np.mean(arr_normal_cdh_loss) =  0.41343766954634614
np.mean(retrieval_N_EAC_adapt_loss) =  0.3760782104252744
np.mean(C2C_EAC_NN_CDH_loss) =  0.523517382413088

reporting k Folds balanced
np.mean(arr_dl) =  0.23369949465188347
np.mean(arr_knn) =  0.29233955656831256
np.mean(arr_retrieval_loss) =  0.41643922242289994
np.mean(arr_retrieval_N_adapt_loss) =  0.22825821933393803
np.mean(arr_normal_cdh_loss) =  0.26221174928319757
np.mean(retrieval_N_EAC_adapt_loss) =  0.2924976075057458
np.mean(C2C_EAC_NN_CDH_loss) =  0.23889882416820804
```





```
reporting k Folds average
np.mean(arr_dl) =  0.5763580401485748
np.mean(arr_knn) =  0.5425783564959727
np.mean(arr_retrieval_loss) =  0.6569103960602646
np.mean(arr_retrieval_N_adapt_loss) =  0.5504342473185594
np.mean(arr_normal_cdh_loss) =  0.46938107758440795
np.mean(retrieval_N_EAC_adapt_loss) =  0.6035167563958099
np.mean(C2C_EAC_NN_CDH_loss) =  0.6354741037519303

reporting std average
np.std(arr_dl) =  0.02179267686442935
np.std(arr_knn) =  0.02165176555679546
np.std(arr_retrieval_loss) =  0.022611759724934604
np.std(arr_retrieval_N_adapt_loss) =  0.024432698325331216
np.std(arr_normal_cdh_loss) =  0.01985915156665192
np.std(retrieval_N_EAC_adapt_loss) =  0.018426137178453882
np.std(C2C_EAC_NN_CDH_loss) =  0.026971598082991023

reporting k Folds balanced
np.mean(arr_dl) =  0.3205154850474546
np.mean(arr_knn) =  0.3526121897166673
np.mean(arr_retrieval_loss) =  0.45766137146761016
np.mean(arr_retrieval_N_adapt_loss) =  0.3313981687459058
np.mean(arr_normal_cdh_loss) =  0.29467081156585084
np.mean(retrieval_N_EAC_adapt_loss) =  0.43030951394650324
np.mean(C2C_EAC_NN_CDH_loss) =  0.43502942456604626

reporting std balanced
np.std(arr_dl) =  0.03753660412843252
np.std(arr_knn) =  0.07687480684309066
np.std(arr_retrieval_loss) =  0.0630762456599508
np.std(arr_retrieval_N_adapt_loss) =  0.04080097523486786
np.std(arr_normal_cdh_loss) =  0.02631982162006131
np.std(retrieval_N_EAC_adapt_loss) =  0.06001922188242892
np.std(C2C_EAC_NN_CDH_loss) =  0.054356441048081613
```

